Authors: José Raúl Romero (jrromero@uco.es), Aurora Ramírez (aurora.ramirez@uma.es), Francisco Javier Alcaide (f52almef@uco.es)

**Multiclass classification problem notebook for UML data set**

- The "dummies" and "unknown" cases will be ignored.

- SVC, RF and KNN have been used

- For the metrics we consider the "macro" approach with the aim that the metrics are not affected by their number.

- It has been verified that after 10 classes there is a higher decrease in performance.

- It has been proven that working with the 10 most frequent classes and deleting the rest results in better performance than grouping the rest into a single class.

- Explainability, the 5 classes with the best performance in F1-score. The instances that have obtained the highest probability for that class were studied.

- Local explainability: 10 features with the greatest contribution in each technique have been considered. And for the coincidence metrics between techniques, the TOP5 has been considered.

# Installation:

It is important to define the path to the folder that contains the Modelset files, in this notebook using the variable "MODELSET_HOME".

In [ ]:
MODELSET_HOME = ""

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install kaleido

In [ ]:
!pip install modelset-py

In [ ]:
!pip install dalex -U

In [ ]:
!pip install lime

In [ ]:
import sys
import pandas as pd
import re
import plotly
import kaleido
import pickle
import numpy as np
import dalex as dx
import os
import modelset.dataset as ds
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, ConfusionMatrixDisplay, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.inspection import permutation_importance

## Functions:

In [ ]:
# Function to evaluate the sign of values
def evaluate_value(value):
    if value >= 0:
        return "Positive"
    else:
        return "Negative"

In [ ]:
# Create the MultiIndex columns
columns_multi = pd.MultiIndex.from_tuples([
    ('Breakdown', 'Ranking'), ('Breakdown', 'Sign'),
    ('Shap', 'Ranking'), ('Shap', 'Sign'),
    ('Lime', 'Ranking'), ('Lime', 'Sign')
])

In [ ]:
# Function to evaluate the top 5 features
def top5_features(df, case):
    set_expl = {}

    for tech in ['Breakdown', 'Shap', 'Lime']:
        inst_top5 = set(df[df[(tech, 'Ranking')].isin([1, 2, 3, 4, 5])].index)
        set_expl[tech] = inst_top5

    result_case = {
        f'TOP5 {tech}': ', '.join(set_expl[tech]) for tech in ['Breakdown', 'Shap', 'Lime']
    }

    for tech1, inst_top5_1 in set_expl.items():
        for tech2, inst_top5_2 in set_expl.items():
            if tech1 < tech2:
                col_name = f'{tech1}-{tech2}'
                result_case[col_name] = len(inst_top5_1.intersection(inst_top5_2)) / 5

    df_top5_case = pd.DataFrame(result_case, index=[case])

    return df_top5_case

In [ ]:
# Function to evaluate the top 5 ranking
def top5_rank(df, case):
    set_expl = {}

    for tech in ['Breakdown', 'Shap', 'Lime']:
        inst_top5 = df[df[(tech, 'Ranking')].isin([1, 2, 3, 4, 5])]
        inst_top5_str = ', '.join([f'{ranking}:{inst}' for ranking, inst in zip(inst_top5[(tech, 'Ranking')], inst_top5.index)])

        set_expl[tech] = inst_top5_str

    result_case = {
        f'TOP5 {tech}': set_expl[tech] for tech in ['Breakdown', 'Shap', 'Lime']
    }

    for tech1, inst_top5_1 in set_expl.items():
        for tech2, inst_top5_2 in set_expl.items():
            if tech1 < tech2:
                col_name = f'{tech1}-{tech2}'

                coincidence = [inst for inst in inst_top5_1.split(', ') if inst in inst_top5_2.split(', ') and inst.split(':')[1] == inst.split(':')[1]]
                num_coincidence = len(coincidence)

                result_case[col_name] = num_coincidence / 5

    df_top5_rank = pd.DataFrame(result_case, index=[case])

    return df_top5_rank

In [ ]:
# Function to evaluate the top 5 sign
def top5_sign(df, case):
    set_expl = {}

    for tech in ['Breakdown', 'Shap', 'Lime']:
        inst_top5 = df[df[(tech, 'Ranking')].isin([1, 2, 3, 4, 5])]
        inst_top5_str = ', '.join([f'{inst}:{sign}' for inst, sign in zip(inst_top5.index, inst_top5[(tech, 'Sign')])])

        set_expl[tech] = inst_top5_str

    result_case = {
        f'TOP5 {tech}': set_expl[tech] for tech in ['Breakdown', 'Shap', 'Lime']
    }

    for tech1, inst_top5_1 in set_expl.items():
        for tech2, inst_top5_2 in set_expl.items():
            if tech1 < tech2:
                col_name = f'{tech1}-{tech2}'

                coincidence = [inst for inst in inst_top5_1.split(', ') if inst in inst_top5_2.split(', ')]
                num_coincidence = len(coincidence)

                result_case[col_name] = num_coincidence / 5

    df_top5_sign = pd.DataFrame(result_case, index=[case])

    return df_top5_sign

In [ ]:
# Function to evaluate the top 5 sign and ranking
def top5_rank_sign(df, case):
    set_expl = {}

    for tech in ['Breakdown', 'Shap', 'Lime']:
        inst_top5 = df[df[(tech, 'Ranking')].isin([1, 2, 3, 4, 5])]
        inst_top5_str = ', '.join([f'{ranking}:{inst}:{sign}' for ranking, inst, sign in zip(inst_top5[(tech, 'Ranking')], inst_top5.index, inst_top5[(tech, 'Sign')])])
        set_expl[tech] = inst_top5_str

    result_case = {
        f'TOP5 {tech}': set_expl[tech] for tech in ['Breakdown', 'Shap', 'Lime']
    }

    for tech1, inst_top5_1 in set_expl.items():
        for tech2, inst_top5_2 in set_expl.items():
            if tech1 < tech2:
                col_name = f'{tech1}-{tech2}'

                coincidence = [inst for inst in inst_top5_1.split(', ') if inst in inst_top5_2.split(', ') and inst.split(':')[1] == inst.split(':')[1] and inst.split(':')[2] == inst.split(':')[2]]
                num_coincidence = len(coincidence)
                result_case[col_name] = num_coincidence / 5

    df_top5_rank_sign = pd.DataFrame(result_case, index=[case])

    return df_top5_rank_sign

In [ ]:
# Function to obtain the dataframes of the tech Breakdown, SHAP, and LIME.
def preprocess_dfs(breakdown_df, shap_df, lime_df, n, m):

    pattern = re.compile(r'(?:<=|<|>=|>)\s*(\w+)\s*(?:<=|<|>=|>)')

    breakdown_df = breakdown_df.loc[:, ['variable_name', 'contribution', 'sign']]
    breakdown_df = breakdown_df.drop(index=[0, n+1])
    breakdown_df['sign'] = breakdown_df['sign'].replace({1.0: 'Positive', 0.0: 'Null', -1.0: 'Negative'})
    breakdown_df = breakdown_df.sort_values(by='contribution', key=lambda x: abs(x), ascending=False)

    shap_df = shap_df.loc[:, ['variable_name', 'contribution', 'sign']]
    shap_df = shap_df.tail(n)
    shap_df['sign'] = shap_df['sign'].replace({1.0: 'Positive', 0.0: 'Null', -1.0: 'Negative'})
    shap_df = shap_df.sort_values(by='contribution', key=lambda x: abs(x), ascending=False)

    lime_list = []
    for feature in lime_df['variable']:
      if pattern.findall(feature):
        match = pattern.search(feature)
        if match:
          lime_list.append(match.group(1))
      else:
        splits = feature.split(" ")
        lime_list.append(splits[0])

    lime_df["Feature"] = lime_list
    lime_df["Sign"] = lime_df["effect"].apply(evaluate_value)
    lime_df = lime_df.sort_values(by='effect', key=lambda x: abs(x), ascending=False)
    lime_df = lime_df.drop(columns=['variable'])

    breakdown_df['Ranking'] = breakdown_df['contribution'].abs().rank(ascending=False).astype(int)
    breakdown_df.rename(columns={'sign': 'Sign', 'variable_name':'Feature'}, inplace=True)
    breakdown_df = breakdown_df[['Feature', 'Ranking', 'contribution', 'Sign']]

    shap_df['Ranking'] = shap_df['contribution'].abs().rank(ascending=False).astype(int)
    shap_df.rename(columns={'sign': 'Sign', 'variable_name':'Feature'}, inplace=True)
    shap_df = shap_df[['Feature', 'Ranking', 'contribution', 'Sign']]

    lime_df['Ranking'] = lime_df['effect'].abs().rank(ascending=False).astype(int)
    lime_df = lime_df.head(m)
    lime_df = lime_df[['Feature', 'Ranking', 'effect', 'Sign']]

    breakdown_df = breakdown_df.drop(columns=['contribution'])
    shap_df = shap_df.drop(columns=['contribution'])
    lime_df = lime_df.drop(columns=['effect'])

    breakdown_df = breakdown_df.head(m)
    breakdown_df = breakdown_df.reset_index(drop=True)

    shap_df= shap_df.head(m)
    shap_df = shap_df.reset_index(drop=True)

    lime_df = lime_df.reset_index(drop=True)

    breakdown_features = list(breakdown_df['Feature'])
    shap_features = list(shap_df['Feature'])
    lime_features = list(lime_df['Feature'])
    all_features = list(set(breakdown_features + shap_features + lime_features))
    all_features = all_features[::-1]

    df_final = pd.DataFrame(index=range(len(all_features)), columns=columns_multi)
    df_final['Feature'] = list(all_features)
    df_final = df_final[['Feature', 'Breakdown', 'Shap', 'Lime']]

    #print(breakdown_features)
    #print(shap_features)
    #print(lime_features)
    print(all_features)

    for feature in df_final['Feature']:

        breakdown_row = breakdown_df[breakdown_df['Feature'] == feature]
        if not breakdown_row.empty:

            ranking_breakdown = breakdown_row.iloc[0]['Ranking']
            sign_breakdown = breakdown_row.iloc[0]['Sign']
            df_final.loc[df_final['Feature'] == feature, ('Breakdown', 'Ranking')] = ranking_breakdown
            df_final.loc[df_final['Feature'] == feature, ('Breakdown', 'Sign')] = sign_breakdown
        else:

            df_final.loc[df_final['Feature'] == feature, ('Breakdown', 'Ranking')] = '-'
            df_final.loc[df_final['Feature'] == feature, ('Breakdown', 'Sign')] = '-'


        shap_row = shap_df[shap_df['Feature'] == feature]
        if not shap_row.empty:
            ranking_shap = shap_row.iloc[0]['Ranking']
            sign_shap = shap_row.iloc[0]['Sign']
            df_final.loc[df_final['Feature'] == feature, ('Shap', 'Ranking')] = ranking_shap
            df_final.loc[df_final['Feature'] == feature, ('Shap', 'Sign')] = sign_shap
        else:
            df_final.loc[df_final['Feature'] == feature, ('Shap', 'Ranking')] = '-'
            df_final.loc[df_final['Feature'] == feature, ('Shap', 'Sign')] = '-'


        lime_row = lime_df[lime_df['Feature'] == feature]
        if not lime_row.empty:
            ranking_lime = lime_row.iloc[0]['Ranking']
            sign_lime = lime_row.iloc[0]['Sign']
            df_final.loc[df_final['Feature'] == feature, ('Lime', 'Ranking')] = ranking_lime
            df_final.loc[df_final['Feature'] == feature, ('Lime', 'Sign')] = sign_lime
        else:
            df_final.loc[df_final['Feature'] == feature, ('Lime', 'Ranking')] = '-'
            df_final.loc[df_final['Feature'] == feature, ('Lime', 'Sign')] = '-'


    ranking_breakdown = df_final[('Breakdown', 'Ranking')]
    values_valid = ranking_breakdown[ranking_breakdown != '-']
    cases_invalid = ranking_breakdown[ranking_breakdown == '-'].index
    cases_valid = []
    cases_invalid_restantes = list(cases_invalid)
    cases_valid.extend(values_valid.sort_values().index.tolist())

    for cas in cases_invalid:
        value_shap = df_final.at[cas, ('Shap', 'Ranking')]
        if value_shap != '-':
            cases_valid.extend([cas])
            cases_invalid_restantes.remove(cas)

    for cas in cases_invalid_restantes:
        value_lime = df_final.at[cas, ('Lime', 'Ranking')]
        if value_lime != '-':
            cases_valid.extend([cas])

    df_final = df_final.loc[cases_valid]
    df_final.set_index('Feature', inplace=True)
    return breakdown_df, shap_df, lime_df, df_final


In [ ]:
# Function to create final dataframes with all the disagreement metrics and calculate the means of disagreement metrics for each pair of techniques.
def calculate_metrics(df_list):
    df_final = pd.concat(df_list)

    mean_bd_shap = df_final['Breakdown-Shap'].mean()
    mean_bd_lime = df_final['Breakdown-Lime'].mean()
    mean_lime_shap = df_final['Lime-Shap'].mean()

    return df_final, mean_bd_shap, mean_bd_lime, mean_lime_shap

In [ ]:
# Function to create dictionaries containing the disagreement metrics dataframes
def create_metrics_dfs(metrics_dict, metric_type):
    dfs = {'Model_comparison': {}, 'Class_comparison': {}}
    columns_model = ['Class', 'Mean Breakdown-Shap', 'Mean Breakdown-Lime', 'Mean Lime-Shap']
    columns_class = ['Model', 'Mean Breakdown-Shap', 'Mean Breakdown-Lime', 'Mean Lime-Shap']
    metric_names = [f'Mean_{metric_type}_BD_Shap', f'Mean_{metric_type}_BD_Lime', f'Mean_{metric_type}_Lime_Shap']

    for model_name, indexes_dict in metrics_dict.items():
        for index_name, metrics in indexes_dict.items():
            if model_name not in dfs['Class_comparison']:
                dfs['Class_comparison'][model_name] = pd.DataFrame(columns=columns_model)

            get_metrics = [metrics.get(metric_name) for metric_name in metric_names]
            class_data = [index_name] + get_metrics
            dfs['Class_comparison'][model_name] = pd.concat([dfs['Class_comparison'][model_name], pd.Series(class_data, index=columns_model).to_frame().T], ignore_index=True)

            if index_name not in dfs['Model_comparison']:
                dfs['Model_comparison'][index_name] = pd.DataFrame(columns=columns_class)

            get_metrics = [metrics.get(metric_name) for metric_name in metric_names]
            model_data = [model_name] + get_metrics
            dfs['Model_comparison'][index_name] = pd.concat([dfs['Model_comparison'][index_name], pd.Series(model_data, index=columns_class).to_frame().T], ignore_index=True)

    return dfs

In [ ]:
# Function to obtain dataframes of explanations, disagreement metrics, and each of the disagreement metrics dataframes
def calculate_metrics_for_indices(models_dict, indexes_dict, X_test, X_train, y_train, num_features, top_num_features):
    results_dict = {}
    metrics_dict = {}

    for model_name, model in models_dict.items():
        results_dict[model_name] = {}
        metrics_dict[model_name] = {}

        exp = dx.Explainer(model, X_train, y_train)

        for index_name, indices in indexes_dict.items():
          results_dict[model_name][index_name] = {}
          metrics_dict[model_name][index_name] = {}
          model_results = results_dict[model_name][index_name]
          df_list_top = []
          df_list_rank = []
          df_list_sign = []
          df_list_rank_sign = []

          for i in indices:
            instance = X_test.loc[i]

            breakdown = exp.predict_parts(instance, type="break_down", random_state=42)
            shap = exp.predict_parts(instance, type="shap", random_state=42)
            lime = exp.predict_surrogate(instance, random_state=42)
            breakdown_df = breakdown.result
            shap_df = shap.result
            lime_df = lime.result

            breakdown_df, shap_df, lime_df, df_final = preprocess_dfs(breakdown_df, shap_df, lime_df, num_features, top_num_features)

            model_results[f"breakdown_df_{i}"] = breakdown_df
            model_results[f"shap_df_{i}"] = shap_df
            model_results[f"lime_df_{i}"] = lime_df
            model_results[f"df_final_{i}"] = df_final

            metrics_dict[model_name][index_name][f"df_top_metric_{i}"] = top3_features(df_final, i)
            metrics_dict[model_name][index_name][f'df_rank_metric_{i}'] = top3_rank(df_final, i)
            metrics_dict[model_name][index_name][f'df_sign_metric_{i}'] = top3_sign(df_final, i)
            metrics_dict[model_name][index_name][f'df_rank_sign_metric_{i}'] = top3_rank_sign(df_final, i)

            #model_results[f"df_top_metric_{i}"] = top3_features(df_final, i)
            #model_results[f'df_rank_metric_{i}'] = top3_rank(df_final, i)
            #model_results[f'df_sign_metric_{i}'] = top3_sign(df_final, i)
            #model_results[f'df_rank_sign_metric_{i}'] = top3_rank_sign(df_final, i)

            df_list_top.append(metrics_dict[model_name][index_name][f'df_top_metric_{i}'])
            df_list_rank.append(metrics_dict[model_name][index_name][f'df_rank_metric_{i}'])
            df_list_sign.append(metrics_dict[model_name][index_name][f'df_sign_metric_{i}'])
            df_list_rank_sign.append(metrics_dict[model_name][index_name][f'df_rank_sign_metric_{i}'])

          df_top_metric_final, mean_top_bd_shap, mean_top_bd_lime, mean_top_lime_shap = calculate_metrics(df_list_top)
          df_rank_metric_final, mean_rank_bd_shap, mean_rank_bd_lime, mean_rank_lime_shap = calculate_metrics(df_list_rank)
          df_sign_metric_final, mean_sign_bd_shap, mean_sign_bd_lime, mean_sign_lime_shap = calculate_metrics(df_list_sign)
          df_rank_sign_metric_final, mean_rank_sign_bd_shap, mean_rank_sign_bd_lime, mean_rank_sign_lime_shap = calculate_metrics(df_list_rank_sign)

          metrics_dict[model_name][index_name][f'Mean_top_BD_Shap'] = mean_top_bd_shap
          metrics_dict[model_name][index_name][f'Mean_top_BD_Lime'] = mean_top_bd_lime
          metrics_dict[model_name][index_name][f'Mean_top_Lime_Shap'] = mean_top_lime_shap
          metrics_dict[model_name][index_name][f'df_top_metric_final'] = df_top_metric_final

          metrics_dict[model_name][index_name][f'Mean_rank_BD_Shap'] = mean_rank_bd_shap
          metrics_dict[model_name][index_name][f'Mean_rank_BD_Lime'] = mean_rank_bd_lime
          metrics_dict[model_name][index_name][f'Mean_rank_Lime_Shap'] = mean_rank_lime_shap
          metrics_dict[model_name][index_name][f'df_rank_metric_final'] = df_rank_metric_final

          metrics_dict[model_name][index_name][f'Mean_sign_BD_Shap'] = mean_sign_bd_shap
          metrics_dict[model_name][index_name][f'Mean_sign_BD_Lime'] = mean_sign_bd_lime
          metrics_dict[model_name][index_name][f'Mean_sign_Lime_Shap'] = mean_sign_lime_shap
          metrics_dict[model_name][index_name][f'df_sign_metric_final'] = df_sign_metric_final

          metrics_dict[model_name][index_name][f'Mean_rank_sign_BD_Shap'] = mean_rank_sign_bd_shap
          metrics_dict[model_name][index_name][f'Mean_rank_sign_BD_Lime'] = mean_rank_sign_bd_lime
          metrics_dict[model_name][index_name][f'Mean_rank_sign_Lime_Shap'] = mean_rank_sign_lime_shap
          metrics_dict[model_name][index_name][f'df_rank_sign_metric_final'] = df_rank_sign_metric_final

    top_metric_df = create_metrics_dfs(metrics_dict, 'top')
    rank_metric_df = create_metrics_dfs(metrics_dict, 'rank')
    sign_metric_df = create_metrics_dfs(metrics_dict, 'sign')
    rank_sign_metric_df = create_metrics_dfs(metrics_dict, 'rank_sign')

    return metrics_dict, results_dict, top_metric_df, rank_metric_df, sign_metric_df, rank_sign_metric_df


# Load y Preprocess:

In this section, we will perform data loading, cleaning, preprocessing, and an initial data analysis.

In [ ]:
dataset = ds.load(MODELSET_HOME, modeltype='uml', selected_analysis=['stats'])
modelset_df = dataset._Dataset__to_df()

duplicates = modelset_df.duplicated(subset="id", keep=False)
inst_dup = modelset_df[duplicates]
inst_dup

In [ ]:
modelset_df

In [ ]:
modelset_df.info()

In [ ]:
# Reset index
modelset_df = modelset_df.reset_index(drop=True)

# Remove useless features
modelset_df = modelset_df.drop(['diagram_ad'], axis=1)
modelset_df = modelset_df.drop(['diagram_cd'], axis=1)
modelset_df = modelset_df.drop(['diagram_usecase'], axis=1)
modelset_df = modelset_df.drop(['diagram_comp'], axis=1)
modelset_df = modelset_df.drop(['diagram_interaction'], axis=1)
modelset_df = modelset_df.drop(['diagram_sm'], axis=1)
modelset_df = modelset_df.drop(['elements'], axis=1)
modelset_df = modelset_df.drop(['tags'], axis=1)
modelset_df = modelset_df.drop(['language'], axis=1)
modelset_df = modelset_df.drop(['id'], axis=1)

In [ ]:
modelset_df.head()

In [ ]:
modelset_df['category'].value_counts()

In [ ]:
modelset_df.describe()

In [ ]:
# Remove constant features
modelset_df = modelset_df.drop(['type_Interaction'], axis=1)
modelset_df = modelset_df.drop(['type_Generalization'], axis=1)
modelset_df = modelset_df.drop(['type_Association'], axis=1)

col = modelset_df.columns
col = col.drop('category')
col

In [ ]:
modelset_df.describe()

In [ ]:
modelset_df_filt = modelset_df.loc[(modelset_df['category'] != 'dummy') & (modelset_df['category'] != 'unknown')] # delete dummy and unknown cases
modelset_df_filt

In [ ]:
modelset_df_filt.isnull().any()

In [ ]:
modelset_df_filt['category'].value_counts()

In [ ]:
# Class distribution
class_counts = modelset_df_filt['category'].value_counts().sort_values(ascending=False).head(30)
plt.figure(figsize=(22, 10))
sns.countplot(x='category', data=modelset_df_filt, order=class_counts.index, palette='viridis')
plt.title('Class Distribution')
plt.xticks(rotation=90)
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(16, 10))
axes = axes.flatten()
for i, feature in enumerate(col):
    sns.histplot(modelset_df_filt[feature], kde=True, bins=30, ax=axes[i])
    axes[i].set_title(f'Distribution {feature}')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation
correlation_matrix = modelset_df_filt.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Corretation Matrix - Model Classification (UML)')
plt.show()



---

❗❗ **No features with a correlation value greater than 0.8, so we are going to keep them all.**


---



# Performance No. Class:

We search for the best number of classes to consider through a performance evaluation

In [ ]:
modelset_df_label = modelset_df_filt.copy()

# transform classes into numbers
label_encoder_nclasses = LabelEncoder()
modelset_df_label['label'] = label_encoder_nclasses.fit_transform(modelset_df_label['category'])

In [ ]:
modelset_df_label.info()

In [ ]:
modelset_df_label.head()

In [ ]:
modelset_df_label = modelset_df_label.drop(['category'], axis=1)

In [ ]:
X_label = modelset_df_label[col]
y_label = modelset_df_label['label']

In [ ]:
nclasses = [10, 20, 30, 40, 50, 60, 70, 80]
results_df = pd.DataFrame(columns=['No. Classes', 'Acc (SVC)', 'Prec (SVC)', 'Rec (SVC)', 'F1 (SVC)',
                                      'Acc (RF)', 'Prec (RF)', 'Rec (RF)', 'F1 (RF)',
                                      'Acc (KNN)', 'Prec (KNN)', 'Rec (KNN)', 'F1 (KNN)'])

for num_classes in nclasses:
    classes_frec_label = y_label.value_counts().nlargest(num_classes).index
    datos_filt = modelset_df_label[modelset_df_label['label'].isin(classes_frec_label)]

    X_filt = datos_filt[col]
    y_filt = datos_filt['label']
    X_train_filt, X_test_filt, y_train_filt, y_test_filt = train_test_split(X_filt, y_filt, test_size=0.3, random_state=42, stratify=y_filt)

    svc_model = SVC(random_state=42)
    rf_model = RandomForestClassifier(random_state=42)
    knn_model = KNeighborsClassifier()

    svc_model.fit(X_train_filt, y_train_filt)
    rf_model.fit(X_train_filt, y_train_filt)
    knn_model.fit(X_train_filt, y_train_filt)

    y_pred_svc = svc_model.predict(X_test_filt)
    y_pred_rf = rf_model.predict(X_test_filt)
    y_pred_knn = knn_model.predict(X_test_filt)

    acc_svc = balanced_accuracy_score(y_test_filt, y_pred_svc)
    precision_svc = precision_score(y_test_filt, y_pred_svc, average='macro')
    recall_svc = recall_score(y_test_filt, y_pred_svc, average='macro')
    f1_svc = f1_score(y_test_filt, y_pred_svc, average='macro')

    acc_rf = balanced_accuracy_score(y_test_filt, y_pred_rf)
    precision_rf = precision_score(y_test_filt, y_pred_rf, average='macro')
    recall_rf = recall_score(y_test_filt, y_pred_rf, average='macro')
    f1_rf = f1_score(y_test_filt, y_pred_rf, average='macro')

    acc_knn = balanced_accuracy_score(y_test_filt, y_pred_knn)
    precision_knn = precision_score(y_test_filt, y_pred_knn, average='macro')
    recall_knn = recall_score(y_test_filt, y_pred_knn, average='macro')
    f1_knn = f1_score(y_test_filt, y_pred_knn, average='macro')

    results_df = results_df.append({
        'No. Classes': num_classes,
        'Acc (SVC)': acc_svc, 'Prec (SVC)': precision_svc, 'Rec (SVC)': recall_svc, 'F1 (SVC)': f1_svc,
        'Acc (RF)': acc_rf, 'Prec (RF)': precision_rf, 'Rec (RF)': recall_rf, 'F1 (RF)': f1_rf,
        'Acc (KNN)': acc_knn, 'Prec (KNN)': precision_knn, 'Rec (KNN)': recall_knn, 'F1 (KNN)': f1_knn}, ignore_index=True)

In [ ]:
results_df


---

❗❗ **We verify that the greatest difference in performance occurs between the cases with 10 and 20 most frequent classes. Therefore, we are going to consider that the best choice is 10 classes.**

---



# Performance Deleting VS Grouping:


Once we have determined the optimal number of classes to consider, we will explore which approach is better for handling the remaining classes: eliminating or grouping.

In [ ]:
num_classes = 10 # optimal number

In [ ]:
modelset_df_group = modelset_df_filt.copy()
X_group = modelset_df_group[col]
y_group = modelset_df_group['category']
classes_frec_group = y_group.value_counts().nlargest(num_classes).index

label_encoder_group = LabelEncoder()
modelset_df_group['category_group'] = modelset_df_group['category'].apply(lambda x: x if x in classes_frec_group else 'Others')
modelset_df_group['label'] = label_encoder_group.fit_transform(modelset_df_group['category_group'])

In [ ]:
classes_frec_group

In [ ]:
modelset_df_group['category_group'].value_counts()

In [ ]:
modelset_df_group

In [ ]:
modelset_df_group = modelset_df_group.drop(['category'], axis=1)
modelset_df_group = modelset_df_group.drop(['category_group'], axis=1)

In [ ]:
classes_frec_label = y_label.value_counts().nlargest(num_classes).index
datos_filt_del = modelset_df_label[modelset_df_label['label'].isin(classes_frec_label)]

In [ ]:
results_df_10 = pd.DataFrame(columns=['Approach', 'Acc (SVC)', 'Prec (SVC)', 'Rec (SVC)', 'F1 (SVC)',
                                      'Acc (RF)', 'Prec (RF)', 'Rec (RF)', 'F1 (RF)',
                                      'Acc (KNN)', 'Prec (KNN)', 'Rec (KNN)', 'F1 (KNN)'])

In [ ]:
X_filt = datos_filt_del[col]
y_filt = datos_filt_del['label']
X_train_filt, X_test_filt, y_train_filt, y_test_filt = train_test_split(X_filt, y_filt, test_size=0.3, random_state=42, stratify=y_filt)

svc_model = SVC(random_state=42)
rf_model = RandomForestClassifier(random_state=42)
knn_model = KNeighborsClassifier()

svc_model.fit(X_train_filt, y_train_filt)
rf_model.fit(X_train_filt, y_train_filt)
knn_model.fit(X_train_filt, y_train_filt)

y_pred_svc_filt = svc_model.predict(X_test_filt)
y_pred_rf_filt = rf_model.predict(X_test_filt)
y_pred_knn_filt = knn_model.predict(X_test_filt)

acc_svc_del = balanced_accuracy_score(y_test_filt, y_pred_svc_filt)
precision_svc_del = precision_score(y_test_filt, y_pred_svc_filt, average='macro')
recall_svc_del = recall_score(y_test_filt, y_pred_svc_filt, average='macro')
f1_svc_del = f1_score(y_test_filt, y_pred_svc_filt, average='macro')

acc_rf_del = balanced_accuracy_score(y_test_filt, y_pred_rf_filt)
precision_rf_del = precision_score(y_test_filt, y_pred_rf_filt, average='macro')
recall_rf_del = recall_score(y_test_filt, y_pred_rf_filt, average='macro')
f1_rf_del = f1_score(y_test_filt, y_pred_rf_filt, average='macro')

acc_knn_del = balanced_accuracy_score(y_test_filt, y_pred_knn_filt)
precision_knn_del = precision_score(y_test_filt, y_pred_knn_filt, average='macro')
recall_knn_del = recall_score(y_test_filt, y_pred_knn_filt, average='macro')
f1_knn_del = f1_score(y_test_filt, y_pred_knn_filt, average='macro')

results_df_10 = results_df_10.append({
    'Approach': 'Deleting',
    'Acc (SVC)': acc_svc_del, 'Prec (SVC)': precision_svc_del, 'Rec (SVC)': recall_svc_del, 'F1 (SVC)': f1_svc_del,
    'Acc (RF)': acc_rf_del, 'Prec (RF)': precision_rf_del, 'Rec (RF)': recall_rf_del, 'F1 (RF)': f1_rf_del,
    'Acc (KNN)': acc_knn_del, 'Prec (KNN)': precision_knn_del, 'Rec (KNN)': recall_knn_del, 'F1 (KNN)': f1_knn_del}, ignore_index=True)

In [ ]:
X_group = modelset_df_group[col]
y_group = modelset_df_group['label']
X_train_group, X_test_group, y_train_group, y_test_group = train_test_split(X_group, y_group, test_size=0.3, random_state=42, stratify=y_group)

svc_model_group = SVC(random_state=42)
rf_model_group = RandomForestClassifier(random_state=42)
knn_model_group = KNeighborsClassifier()

svc_model_group.fit(X_train_group, y_train_group)
rf_model_group.fit(X_train_group, y_train_group)
knn_model_group.fit(X_train_group, y_train_group)

y_pred_svc_group = svc_model_group.predict(X_test_group)
y_pred_rf_group = rf_model_group.predict(X_test_group)
y_pred_knn_group = knn_model_group.predict(X_test_group)

acc_svc_group = balanced_accuracy_score(y_test_group, y_pred_svc_group)
precision_svc_group = precision_score(y_test_group, y_pred_svc_group, average='macro')
recall_svc_group = recall_score(y_test_group, y_pred_svc_group, average='macro')
f1_svc_group = f1_score(y_test_group, y_pred_svc_group, average='macro')

acc_rf_group = balanced_accuracy_score(y_test_group, y_pred_rf_group)
precision_rf_group = precision_score(y_test_group, y_pred_rf_group, average='macro')
recall_rf_group = recall_score(y_test_group, y_pred_rf_group, average='macro')
f1_rf_group = f1_score(y_test_group, y_pred_rf_group, average='macro')

acc_knn_group = balanced_accuracy_score(y_test_group, y_pred_knn_group)
precision_knn_group = precision_score(y_test_group, y_pred_knn_group, average='macro')
recall_knn_group = recall_score(y_test_group, y_pred_knn_group, average='macro')
f1_knn_group = f1_score(y_test_group, y_pred_knn_group, average='macro')

results_df_10 = results_df_10.append({
    'Approach': 'Grouping',
    'Acc (SVC)': acc_svc_group, 'Prec (SVC)': precision_svc_group, 'Rec (SVC)': recall_svc_group, 'F1 (SVC)': f1_svc_group,
    'Acc (RF)': acc_rf_group, 'Prec (RF)': precision_rf_group, 'Rec (RF)': recall_rf_group, 'F1 (RF)': f1_rf_group,
    'Acc (KNN)': acc_knn_group, 'Prec (KNN)': precision_knn_group, 'Rec (KNN)': recall_knn_group, 'F1 (KNN)': f1_knn_group}, ignore_index=True)

In [ ]:
results_df_10



---

❗❗ **Performance is greater by eliminating the rest of the classes than by grouping them into a single class**

---

# Models:

In this section, we perform the dataset splitting and data scaling. Afterward, we conduct a search for the best hyperparameters for our algorithms and define them with the best configuration found. Finally, we evaluate the performance.

In [ ]:
X = datos_filt_del[col]
Y = datos_filt_del['label']

X_train_split, X_test_split, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)

In [ ]:
X_train_split

In [ ]:
X_test_split

In [ ]:
X_train_index = X_train_split.index
X_test_index = X_test_split.index

scaler = StandardScaler()

X_train_scal = scaler.fit_transform(X_train_split)
X_train = pd.DataFrame(X_train_scal, columns=X_train_split.columns, index=X_train_index)

X_test_scal = scaler.transform(X_test_split)
X_test = pd.DataFrame(X_test_scal, columns=X_test_split.columns, index=X_test_index)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
#X_train.to_csv('uml_multiclass_x_train.csv')
#X_test.to_csv('uml_multiclass_x_test.csv')
#y_train.to_csv('uml_multiclass_y_train.csv')
#y_test.to_csv('uml_multiclass_y_test.csv')

In [ ]:
score_result = pd.DataFrame(columns=['Model', 'Balanced Accuracy', 'Precision', 'Recall', 'F1-score'])

In [ ]:
classes_frec_label # most frequent Classes

In [ ]:
class_labels = [4, 35, 37, 42, 49, 58, 61, 83, 91, 98]
classes_interes_transf = label_encoder_nclasses.inverse_transform(class_labels)
mapeo_classes_interes = {class_label: number for class_label, number in zip(classes_interes_transf, class_labels)}
print("Mapping classes to numbers:")
for class_label, number in mapeo_classes_interes.items():
    print(f"{class_label}: {number}")

## **SVC**:

In [ ]:
model = SVC(random_state=42, probability=True)
params={'C':[0.1, 1.0, 10.0, 100.0, 300.0, 500.0, 1000.0],'kernel':['linear','rbf','sigmoid'],'gamma':['scale','auto', 0.1, 1.0, 10.0, 100.0, 300.0, 500.0, 1000.0], 'class_weight':[None, 'balanced']}
grid = GridSearchCV(model, params, cv=5, scoring='f1_macro', verbose=1)
grid.fit(X_train,y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

In [ ]:
svc = SVC(C=500.0, gamma=0.1, probability=True, random_state=42)

In [ ]:
model_svc = svc.fit(X_train, y_train)

In [ ]:
y_pred_svc = model_svc.predict(X_test)

**MODEL EVALUATION:**

In [ ]:
df_pred = pd.DataFrame({'Actual': y_test.squeeze(), 'Predicted': y_pred_svc.squeeze()})
print(df_pred)

In [ ]:
cm_svc = confusion_matrix(y_test, y_pred_svc)
cm_display_svc = ConfusionMatrixDisplay(confusion_matrix=cm_svc, display_labels=class_labels)
fig, ax = plt.subplots(figsize=(8, 8))
cm_display_svc.plot(cmap='viridis', values_format='d', ax=ax)
plt.show()

In [ ]:

accuracy = balanced_accuracy_score(y_test, y_pred_svc)
recall = recall_score(y_test, y_pred_svc, average='macro')
precision = precision_score(y_test, y_pred_svc, average = 'macro')
f1 = f1_score(y_test, y_pred_svc, average = 'macro')
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1-score:", f1)

In [ ]:
print("Accuracy test: ", model_svc.score(X_test, y_test))
print("Accuracy train: ", model_svc.score(X_train, y_train))

**CROSS VALIDATION**

In [ ]:
cv_results = cross_validate(svc, X_test, y_test, cv=5, scoring=('balanced_accuracy','precision_macro', 'recall_macro','f1_macro'))

In [ ]:
sorted(cv_results.keys())

In [ ]:
print("Accuracy:\n", cv_results['test_balanced_accuracy'], "\n Mean: ", np.mean(cv_results['test_balanced_accuracy']))
print("Precision:\n" , cv_results['test_precision_macro'], "\n Mean: ", np.mean(cv_results['test_precision_macro']))
print("Recall:\n", cv_results['test_recall_macro'], "\n Mean: ", np.mean(cv_results['test_recall_macro']))
print("F1-score:\n", cv_results['test_f1_macro'], "\n Mean: ", np.mean(cv_results['test_f1_macro']))

In [ ]:
score_result= score_result.append({"Model":"SVC","Balanced Accuracy":np.mean(cv_results['test_balanced_accuracy']), "Precision":np.mean(cv_results['test_precision_macro']), "Recall":np.mean(cv_results['test_recall_macro']), "F1-score":np.mean(cv_results['test_f1_macro'])}, ignore_index=True)

## **RandomForest:**

In [ ]:
model = RandomForestClassifier(random_state=42)
params = {'n_estimators':[50,100,200,300,500],'min_samples_split':[2,3,5,10],'min_samples_leaf':[1,3,5,10],'max_depth':[1,3,5,10,None], 'class_weight':['balanced', None]}
grid = GridSearchCV(model, params, cv=5, scoring='f1_macro', verbose=1)
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

In [ ]:
rf = RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
model_rf = rf.fit(X_train,y_train)

In [ ]:
y_pred_rf = model_rf.predict(X_test)

**MODEL EVALUATION:**

In [ ]:
df_pred = pd.DataFrame({'Actual': y_test.squeeze(), 'Predicted': y_pred_rf.squeeze()})
print(df_pred)

In [ ]:
cm_rf = confusion_matrix(y_test, y_pred_rf)
cm_display_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=class_labels)
fig, ax = plt.subplots(figsize=(8, 8))
cm_display_rf.plot(cmap='viridis', values_format='d', ax=ax)
plt.show()

In [ ]:
accuracy = balanced_accuracy_score(y_test, y_pred_rf)
recall = recall_score(y_test, y_pred_rf, average='macro')
precision = precision_score(y_test, y_pred_rf, average='macro')
f1 = f1_score(y_test, y_pred_rf, average='macro')
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1-score:", f1)

In [ ]:
print("Accuracy test: ", model_rf.score(X_test, y_test))
print("Accuracy train: ", model_rf.score(X_train, y_train))

**CROSS VALIDATION**

In [ ]:
cv_results = cross_validate(rf, X_test, y_test, cv=5, scoring=('balanced_accuracy','precision_macro', 'recall_macro','f1_macro'))

In [ ]:
sorted(cv_results.keys())

In [ ]:
print("Accuracy:\n", cv_results['test_balanced_accuracy'], "\n Mean: ", np.mean(cv_results['test_balanced_accuracy']))
print("Precision:\n" , cv_results['test_precision_macro'], "\n Mean: ", np.mean(cv_results['test_precision_macro']))
print("Recall:\n", cv_results['test_recall_macro'], "\n Mean: ", np.mean(cv_results['test_recall_macro']))
print("F1-score:\n", cv_results['test_f1_macro'], "\n Mean: ", np.mean(cv_results['test_f1_macro']))

In [ ]:
score_result= score_result.append({"Model":"RF","Balanced Accuracy":np.mean(cv_results['test_balanced_accuracy']), "Precision":np.mean(cv_results['test_precision_macro']), "Recall":np.mean(cv_results['test_recall_macro']), "F1-score":np.mean(cv_results['test_f1_macro'])}, ignore_index=True)

## **KNN:**

In [ ]:
model = KNeighborsClassifier()
params = {'n_neighbors':[2,3,4,5,7,10,20],'weights':['uniform','distance'],'leaf_size':[5,10,30,50,100],'p':[1,2,3]}
grid = GridSearchCV(model, params, cv=5, scoring='f1_macro', verbose=1)
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

In [ ]:
knn = KNeighborsClassifier(leaf_size=50, n_neighbors=3, p=1, weights='distance')

In [ ]:
model_knn = knn.fit(X_train,y_train)

In [ ]:
y_pred_knn = model_knn.predict(X_test)

**MODEL EVALUATION:**

In [ ]:
df_pred = pd.DataFrame({'Actual': y_test.squeeze(), 'Predicted': y_pred_knn.squeeze()})
print(df_pred)

In [ ]:
cm_knn = confusion_matrix(y_test, y_pred_knn)
cm_display_knn = ConfusionMatrixDisplay(confusion_matrix=cm_knn, display_labels=class_labels)
fig, ax = plt.subplots(figsize=(8, 8))
cm_display_knn.plot(cmap='viridis', values_format='d', ax=ax)
plt.show()

In [ ]:
accuracy = balanced_accuracy_score(y_test, y_pred_knn)
recall = recall_score(y_test, y_pred_knn, average='macro')
precision = precision_score(y_test, y_pred_knn, average='macro')
f1 = f1_score(y_test, y_pred_knn, average='macro')
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1-score:", f1)

In [ ]:
print("Accuracy test: ", model_knn.score(X_test, y_test))
print("Accuracy train: ", model_knn.score(X_train, y_train))

**CROSS VALIDATION**

In [ ]:
cv_results = cross_validate(knn, X_test, y_test, cv=5, scoring=('balanced_accuracy','precision_macro', 'recall_macro','f1_macro'))

In [ ]:
sorted(cv_results.keys())

In [ ]:
print("Accuracy:\n", cv_results['test_balanced_accuracy'], "\n Mean: ", np.mean(cv_results['test_balanced_accuracy']))
print("Precision:\n" , cv_results['test_precision_macro'], "\n Mean: ", np.mean(cv_results['test_precision_macro']))
print("Recall:\n", cv_results['test_recall_macro'], "\n Mean: ", np.mean(cv_results['test_recall_macro']))
print("F1-score:\n", cv_results['test_f1_macro'], "\n Mean: ", np.mean(cv_results['test_f1_macro']))

In [ ]:
score_result= score_result.append({"Model":"KNN","Balanced Accuracy":np.mean(cv_results['test_balanced_accuracy']), "Precision":np.mean(cv_results['test_precision_macro']), "Recall":np.mean(cv_results['test_recall_macro']), "F1-score":np.mean(cv_results['test_f1_macro'])}, ignore_index=True)

## **Results:**

In [ ]:
score_result

In [ ]:
# plot results
metrics = ['Balanced Accuracy', 'Recall', 'Precision', 'F1-score']
colors = ['#3498DB', '#F7DC6F','#EC7063', '#58D68D']

num_models = len(score_result)
width_bar = 0.20
x = range(num_models)

plt.figure(figsize=(12, 8))

for i, metric in enumerate(metrics):
    pos_x = [pos + i * width_bar for pos in x]
    values_metric = score_result[metric]

    plt.bar(
        pos_x,
        values_metric,
        width=width_bar,
        label=metric,
        color=colors[i],
    )

plt.xlabel('Model')
plt.ylabel('Metric Value')
plt.title('Models performance comparison - Model Classification (UML)')
plt.xticks([pos + (len(metrics) - 1) * width_bar / 2 for pos in x], score_result['Model'], rotation=45, ha="right")
plt.legend(loc='best')
plt.tight_layout()
plt.ylim(0,1)
plt.show()

In [ ]:
y_test.value_counts()

In [ ]:
classes_interes_transf = label_encoder_nclasses.inverse_transform(class_labels)
mapeo_classes_interes = {class_label: number for class_label, number in zip(classes_interes_transf, class_labels)}
print("Mapping classes to numbers:")
for class_label, number in mapeo_classes_interes.items():
    print(f"{class_label}: {number}")

In [ ]:
classifiers = ["SVC", "RandomForest", "KNN"]
matrices_confusion = [cm_svc, cm_rf, cm_knn]
class_names = ["4", "35", "37", "42", "49", "58", "61", "83", "91", "98"]

fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey="row")

for i, (cm, classifier_name) in enumerate(zip(matrices_confusion, classifiers)):

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

    disp.plot(ax=axes[i], cmap="Blues")
    axes[i].set_title(classifier_name)
    disp.im_.colorbar.remove()
    disp.ax_.set_xlabel('')
    if i!=0:
        disp.ax_.set_ylabel('')

plt.tight_layout()
fig.text(0.4, 0.0, "Predicted label", ha="left")
fig.colorbar(disp.im_, ax=axes)
plt.show()

In [ ]:
report_svc = classification_report(y_test, y_pred_svc, target_names=classes_interes_transf)
print(report_svc)

In [ ]:
report_rf = classification_report(y_test, y_pred_rf, target_names=classes_interes_transf)
print(report_rf)

In [ ]:
report_knn = classification_report(y_test, y_pred_knn, target_names=classes_interes_transf)
print(report_knn)



---

The classes with highest F1-score:
- computerarchitecture ---> 37
- restaurant ---> 83
- shopping ---> 91
- employee ---> 49
- smarthouse ---> 98

---



# Instance Selection:

In this section, we perform the selection of instances to be studied in local explainability. The selected instances must meet the condition of being classified in the same class by all three models.

In [ ]:
df_preds = pd.DataFrame({'Real': y_test, 'Pred_SVC': y_pred_svc, 'Pred_RF': y_pred_rf, 'Pred_KNN': y_pred_knn})
df_preds = df_preds[(df_preds['Real'] == df_preds['Pred_SVC']) & (df_preds['Real'] == df_preds['Pred_RF']) & (df_preds['Real'] == df_preds['Pred_KNN'])]

In [ ]:
df_preds

In [ ]:
df_class_comp = df_preds[df_preds['Real'] == 37]
df_class_rest = df_preds[df_preds['Real'] == 83]
df_class_shop = df_preds[df_preds['Real'] == 91]
df_class_emp = df_preds[df_preds['Real'] == 49]
df_class_smart = df_preds[df_preds['Real'] == 98]

In [ ]:
print(f"Number of instances for the class Computerarchitecture: {len(df_class_comp)}")
print(f"Number of instances for the class Restaurant: {len(df_class_rest)}")
print(f"Number of instances for the class Shopping: {len(df_class_shop)}")
print(f"Number of instances for the class Employee: {len(df_class_emp)}")
print(f"Number of instances for the class Smarthouse: {len(df_class_smart)}")

In [ ]:
df_class_comp.head()

In [ ]:
X_preds = X_test.loc[df_preds[df_preds['Real'] == 37].index]
X_preds.head()

In [ ]:
instances_class = df_preds[df_preds['Real'] == 37]
instances_class.head()

In [ ]:
df_class_rest.head()

In [ ]:
df_class_shop.head()

In [ ]:
df_class_emp.head()

In [ ]:
df_class_smart.head()

In [ ]:
dfs_select = ["df_class_comp", "df_class_rest", "df_class_shop", "df_class_emp", "df_class_smart"]
inst_select_indexes = {}

for df_select in dfs_select:
    df = globals()[df_select]
    if len(df) > 100:
        indexes_inst = df.sample(n=100, random_state=42).index
    else:
        indexes_inst = df.index
    indexes_inst = list(indexes_inst)
    inst_select_indexes[df_select] = indexes_inst

In [ ]:
with open('uml_multiclass_indexes.pickle', 'wb') as f:
    pickle.dump(inst_select_indexes, f)

In [ ]:
classes_selected = [37, 83, 91, 49, 98]

for class_label in classes_selected:
    instances_class = df_preds[df_preds['Real'] == class_label]
    X_preds = X_test.loc[df_preds[df_preds['Real'] == class_label].index]

    probab_pred_svc = svc.predict_proba(X_preds)
    probab_pred_rf = rf.predict_proba(X_preds)
    probab_pred_knn = knn.predict_proba(X_preds)

    index_class = np.where(classes_interes_transf ==  label_encoder_nclasses.inverse_transform([class_label]))[0][0]

    prob_svc = probab_pred_svc[:, index_class]
    prob_rf = probab_pred_rf[:, index_class]
    prob_knn = probab_pred_knn[:, index_class]

    index_max_prob_svc = prob_svc.argmax()
    index_max_prob_rf = prob_rf.argmax()
    index_max_prob_knn = prob_knn.argmax()

    instance_max_prob_svc = instances_class.iloc[index_max_prob_svc]
    instance_max_prob_rf = instances_class.iloc[index_max_prob_rf]
    instance_max_prob_knn = instances_class.iloc[index_max_prob_knn]

    if class_label == 37:
      inst_comp_svc = instance_max_prob_svc.name
      inst_comp_rf = instance_max_prob_rf.name
      inst_comp_knn = instance_max_prob_knn.name
    elif class_label == 83:
      inst_rest_svc = instance_max_prob_svc.name
      inst_rest_rf = instance_max_prob_rf.name
      inst_rest_knn = instance_max_prob_knn.name
    elif class_label == 91:
      inst_shop_svc = instance_max_prob_svc.name
      inst_shop_rf = instance_max_prob_rf.name
      inst_shop_knn = instance_max_prob_knn.name
    elif class_label == 49:
      inst_emp_svc = instance_max_prob_svc.name
      inst_emp_rf = instance_max_prob_rf.name
      inst_emp_knn = instance_max_prob_knn.name
    elif class_label == 98:
      inst_smart_svc = instance_max_prob_svc.name
      inst_smart_rf = instance_max_prob_rf.name
      inst_smart_knn = instance_max_prob_knn.name

    print(f"For class {class_label} ({label_encoder_nclasses.inverse_transform([class_label])[0]})")
    print(f"Instance with higher predict_proba SVC: {instance_max_prob_svc.name}")
    print(f"Instance with higher predict_proba RF: {instance_max_prob_rf.name}")
    print(f"Instance with higher predict_proba KNN: {instance_max_prob_knn.name}")
    print("------------------------------------------------------------------------")

# Explainability:

# Global Explainability:

We conduct the study of global explainability using permutation importance.

In [ ]:
permu = permutation_importance(model_svc, X_test, y_test, scoring='f1_macro', n_repeats=20, random_state=42, n_jobs=2)
permu_importances = pd.Series(permu.importances_mean.round(3), index=X.columns)[abs(permu.importances_mean.round(3)) > 0.000]
permu_importances.sort_values(ascending=False, inplace=True)
fig, ax = plt.subplots(figsize=(8,7))
permu_importances.plot.bar(yerr=permu.importances_std[abs(permu.importances_mean.round(3)) > 0.000], ax=ax)
ax.set_ylabel("Mean F1-Score decrease", fontsize=18, labelpad=13)
ax.set_xlabel("Feature", fontsize=18, labelpad=13)
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)
fig.tight_layout()
plt.savefig('uml_multiclass_globSVC.pdf')
plt.show()

In [ ]:
permu = permutation_importance(model_rf, X_test, y_test, scoring='f1_macro', n_repeats=20, random_state=42, n_jobs=2)
permu_importances = pd.Series(permu.importances_mean.round(3), index=X.columns)[abs(permu.importances_mean.round(3)) > 0.000]
permu_importances.sort_values(ascending=False, inplace=True)
fig, ax = plt.subplots(figsize=(8,7))
permu_importances.plot.bar(yerr=permu.importances_std[abs(permu.importances_mean.round(3)) > 0.000], ax=ax)
ax.set_ylabel("Mean F1-Score decrease", fontsize=18, labelpad=13)
ax.set_xlabel("Feature", fontsize=18, labelpad=13)
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)
fig.tight_layout()
plt.savefig('uml_multiclass_globRF.pdf')
plt.show()

In [ ]:
permu = permutation_importance(model_knn, X_test, y_test, scoring='f1_macro', n_repeats=20, random_state=42, n_jobs=2)
permu_importances = pd.Series(permu.importances_mean.round(3), index=X.columns)[abs(permu.importances_mean.round(3)) > 0.000]
permu_importances.sort_values(ascending=False, inplace=True)
fig, ax = plt.subplots(figsize=(8,7))
permu_importances.plot.bar(yerr=permu.importances_std[abs(permu.importances_mean.round(3)) > 0.000], ax=ax)
ax.set_ylabel("Mean F1-Score decrease", fontsize=18, labelpad=13)
ax.set_xlabel("Feature", fontsize=18, labelpad=13)
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)
fig.tight_layout()
plt.savefig('uml_multiclass_globKNN.pdf')
plt.show()

# Local Explainability:

Finally, we obtain explanations for the selected instances and calculate the disagreement metrics.

In [ ]:
num_features = len(col)
top_num_features = 10

In [ ]:
unique_values = np.unique(y_train)
inst_names = ["computerarchitecture", "restaurant", "shopping", "employee", "smarthouse"]
class_index = [2, 4, 7, 8, 9]

## SVC:

In [ ]:
exp_list_svc = []

for i in range(len(unique_values)):
    class_value = unique_values[i]
    pf = lambda m, d, i=i: m.predict_proba(d)[:, i]
    e = dx.Explainer(
        model_svc, X_train,
        y_train,
        predict_function=pf,
        label=f'SVC: class {class_value}',
        verbose=False
    )
    exp_list_svc += [e]

exp_list_svc

In [ ]:
inst_computerarchitecture = X_test.loc[inst_comp_svc]
inst_restaurant = X_test.loc[inst_rest_svc]
inst_shopping = X_test.loc[inst_shop_svc]
inst_employee = X_test.loc[inst_emp_svc]
inst_smarthouse = X_test.loc[inst_smart_svc]

In [ ]:
svc_exp_results = {}

for instance in inst_names:
    breakdown_exp = [exp.predict_parts(eval(f"inst_{instance}"), type='break_down', random_state=42) for exp in exp_list_svc]
    svc_exp_results[f"breakdown_{instance}"] = breakdown_exp

    shap_exp = [exp.predict_parts(eval(f"inst_{instance}"), type='shap', random_state=42) for exp in exp_list_svc]
    svc_exp_results[f"shap_{instance}"] = shap_exp

    lime_exp = [exp.predict_surrogate(eval(f"inst_{instance}"), random_state=42, class_names=[0, unique_values[i]]) for i, exp in enumerate(exp_list_svc)]
    svc_exp_results[f"lime_{instance}"] = lime_exp

### **Computer-architecture:**

In [ ]:
breakdown_df_computerarchitecture_comparch = svc_exp_results['breakdown_computerarchitecture'][2].result
shap_df_computerarchitecture_comparch = svc_exp_results['shap_computerarchitecture'][2].result
lime_df_computerarchitecture_comparch = svc_exp_results['lime_computerarchitecture'][2].result

In [ ]:
breakdown_df_computerarchitecture_emp = svc_exp_results['breakdown_computerarchitecture'][4].result
shap_df_computerarchitecture_emp = svc_exp_results['shap_computerarchitecture'][4].result
lime_df_computerarchitecture_emp = svc_exp_results['lime_computerarchitecture'][4].result

In [ ]:
breakdown_df_computerarchitecture_rest = svc_exp_results['breakdown_computerarchitecture'][7].result
shap_df_computerarchitecture_rest = svc_exp_results['shap_computerarchitecture'][7].result
lime_df_computerarchitecture_rest = svc_exp_results['lime_computerarchitecture'][7].result

In [ ]:
breakdown_df_computerarchitecture_shop = svc_exp_results['breakdown_computerarchitecture'][8].result
shap_df_computerarchitecture_shop = svc_exp_results['shap_computerarchitecture'][8].result
lime_df_computerarchitecture_shop = svc_exp_results['lime_computerarchitecture'][8].result

In [ ]:
breakdown_df_computerarchitecture_smart = svc_exp_results['breakdown_computerarchitecture'][9].result
shap_df_computerarchitecture_smart = svc_exp_results['shap_computerarchitecture'][9].result
lime_df_computerarchitecture_smart = svc_exp_results['lime_computerarchitecture'][9].result

In [ ]:
exp_plots_bd = [svc_exp_results['breakdown_computerarchitecture'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [svc_exp_results['shap_computerarchitecture'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  svc_exp_results['lime_computerarchitecture'][i].plot()

In [ ]:
breakdown_df_computerarchitecture_comparch, shap_df_computerarchitecture_comparch, lime_df_computerarchitecture_comparch, df_final_svc_computerarchitecture_comparch = preprocess_dfs(breakdown_df_computerarchitecture_comparch, 
                                                                                                                                                                                      shap_df_computerarchitecture_comparch, 
                                                                                                                                                                                      lime_df_computerarchitecture_comparch, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_comparch)
print('----------------------------------------------')
print(shap_df_computerarchitecture_comparch)
print('----------------------------------------------')
print(lime_df_computerarchitecture_comparch)

In [ ]:
df_final_svc_computerarchitecture_comparch

In [ ]:
breakdown_df_computerarchitecture_emp, shap_df_computerarchitecture_emp, lime_df_computerarchitecture_emp, df_final_svc_computerarchitecture_emp = preprocess_dfs(breakdown_df_computerarchitecture_emp, 
                                                                                                                                                                                      shap_df_computerarchitecture_emp, 
                                                                                                                                                                                      lime_df_computerarchitecture_emp, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_emp)
print('----------------------------------------------')
print(shap_df_computerarchitecture_emp)
print('----------------------------------------------')
print(lime_df_computerarchitecture_emp)

In [ ]:
df_final_svc_computerarchitecture_emp

In [ ]:
breakdown_df_computerarchitecture_rest, shap_df_computerarchitecture_rest, lime_df_computerarchitecture_rest, df_final_svc_computerarchitecture_rest = preprocess_dfs(breakdown_df_computerarchitecture_rest, 
                                                                                                                                                                                      shap_df_computerarchitecture_rest, 
                                                                                                                                                                                      lime_df_computerarchitecture_rest, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_rest)
print('----------------------------------------------')
print(shap_df_computerarchitecture_rest)
print('----------------------------------------------')
print(lime_df_computerarchitecture_rest)

In [ ]:
df_final_svc_computerarchitecture_rest

In [ ]:
breakdown_df_computerarchitecture_shop, shap_df_computerarchitecture_shop, lime_df_computerarchitecture_shop, df_final_svc_computerarchitecture_shop = preprocess_dfs(breakdown_df_computerarchitecture_shop, 
                                                                                                                                                                                      shap_df_computerarchitecture_shop, 
                                                                                                                                                                                      lime_df_computerarchitecture_shop, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_shop)
print('----------------------------------------------')
print(shap_df_computerarchitecture_shop)
print('----------------------------------------------')
print(lime_df_computerarchitecture_shop)

In [ ]:
df_final_svc_computerarchitecture_shop

In [ ]:
breakdown_df_computerarchitecture_smart, shap_df_computerarchitecture_smart, lime_df_computerarchitecture_smart, df_final_svc_computerarchitecture_smart = preprocess_dfs(breakdown_df_computerarchitecture_smart, 
                                                                                                                                                                                      shap_df_computerarchitecture_smart, 
                                                                                                                                                                                      lime_df_computerarchitecture_smart, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_smart)
print('----------------------------------------------')
print(shap_df_computerarchitecture_smart)
print('----------------------------------------------')
print(lime_df_computerarchitecture_smart)

In [ ]:
df_final_svc_computerarchitecture_smart

### **Results Computer-architecture:**

In [ ]:
df_topfeatures_comparch_comparch = top5_features(df_final_svc_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp = top5_features(df_final_svc_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest = top5_features(df_final_svc_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop = top5_features(df_final_svc_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart = top5_features(df_final_svc_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch, df_topfeatures_comparch_emp, df_topfeatures_comparch_rest, 
                                                 df_topfeatures_comparch_shop, df_topfeatures_comparch_smart])
df_topfeatures_computerarchitecture

In [ ]:
df_topfeatures_comparch_comparch_rank = top5_rank(df_final_svc_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp_rank = top5_rank(df_final_svc_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest_rank = top5_rank(df_final_svc_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop_rank = top5_rank(df_final_svc_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart_rank = top5_rank(df_final_svc_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_rank_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch_rank, df_topfeatures_comparch_emp_rank, 
                                                      df_topfeatures_comparch_rest_rank, df_topfeatures_comparch_shop_rank, df_topfeatures_comparch_smart_rank])
df_topfeatures_rank_computerarchitecture

In [ ]:
df_topfeatures_comparch_comparch_sign = top5_sign(df_final_svc_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp_sign = top5_sign(df_final_svc_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest_sign = top5_sign(df_final_svc_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop_sign = top5_sign(df_final_svc_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart_sign = top5_sign(df_final_svc_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_sign_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch_sign, df_topfeatures_comparch_emp_sign, 
                                                      df_topfeatures_comparch_rest_sign, df_topfeatures_comparch_shop_sign, df_topfeatures_comparch_smart_sign])
df_topfeatures_sign_computerarchitecture

In [ ]:
df_topfeatures_comparch_comparch_rank_sign = top5_rank_sign(df_final_svc_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp_rank_sign = top5_rank_sign(df_final_svc_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest_rank_sign = top5_rank_sign(df_final_svc_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop_rank_sign = top5_rank_sign(df_final_svc_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart_rank_sign = top5_rank_sign(df_final_svc_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_rank_sign_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch_rank_sign, df_topfeatures_comparch_emp_rank_sign, 
                                                           df_topfeatures_comparch_rest_rank_sign, df_topfeatures_comparch_shop_rank_sign, 
                                                           df_topfeatures_comparch_smart_rank_sign])
df_topfeatures_rank_sign_computerarchitecture

### **Employee:**

In [ ]:
breakdown_df_employee_comparch = svc_exp_results['breakdown_employee'][2].result
shap_df_employee_comparch = svc_exp_results['shap_employee'][2].result
lime_df_employee_comparch = svc_exp_results['lime_employee'][2].result

In [ ]:
breakdown_df_employee_emp = svc_exp_results['breakdown_employee'][4].result
shap_df_employee_emp = svc_exp_results['shap_employee'][4].result
lime_df_employee_emp = svc_exp_results['lime_employee'][4].result

In [ ]:
breakdown_df_employee_rest = svc_exp_results['breakdown_employee'][7].result
shap_df_employee_rest = svc_exp_results['shap_employee'][7].result
lime_df_employee_rest = svc_exp_results['lime_employee'][7].result

In [ ]:
breakdown_df_employee_shop = svc_exp_results['breakdown_employee'][8].result
shap_df_employee_shop = svc_exp_results['shap_employee'][8].result
lime_df_employee_shop = svc_exp_results['lime_employee'][8].result

In [ ]:
breakdown_df_employee_smart = svc_exp_results['breakdown_employee'][9].result
shap_df_employee_smart = svc_exp_results['shap_employee'][9].result
lime_df_employee_smart = svc_exp_results['lime_employee'][9].result

In [ ]:
exp_plots_bd = [svc_exp_results['breakdown_employee'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [svc_exp_results['shap_employee'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  svc_exp_results['lime_employee'][i].plot()

In [ ]:
breakdown_df_employee_comparch, shap_df_employee_comparch, lime_df_employee_comparch, df_final_svc_employee_comparch = preprocess_dfs(breakdown_df_employee_comparch, 
                                                                                                                                      shap_df_employee_comparch, 
                                                                                                                                      lime_df_employee_comparch, 
                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_comparch)
print('----------------------------------------------')
print(shap_df_employee_comparch)
print('----------------------------------------------')
print(lime_df_employee_comparch)

In [ ]:
df_final_svc_employee_comparch

In [ ]:
breakdown_df_employee_emp, shap_df_employee_emp, lime_df_employee_emp, df_final_svc_employee_emp = preprocess_dfs(breakdown_df_employee_emp, 
                                                                                                                  shap_df_employee_emp, 
                                                                                                                  lime_df_employee_emp, 
                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_emp)
print('----------------------------------------------')
print(shap_df_employee_emp)
print('----------------------------------------------')
print(lime_df_employee_emp)

In [ ]:
df_final_svc_employee_emp

In [ ]:
breakdown_df_employee_rest, shap_df_employee_rest, lime_df_employee_rest, df_final_svc_employee_rest = preprocess_dfs(breakdown_df_employee_rest, 
                                                                                                                      shap_df_employee_rest, 
                                                                                                                      lime_df_employee_rest, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_rest)
print('----------------------------------------------')
print(shap_df_employee_rest)
print('----------------------------------------------')
print(lime_df_employee_rest)

In [ ]:
df_final_svc_employee_rest

In [ ]:
breakdown_df_employee_shop, shap_df_employee_shop, lime_df_employee_shop, df_final_svc_employee_shop = preprocess_dfs(breakdown_df_employee_shop, 
                                                                                                                      shap_df_employee_shop, 
                                                                                                                      lime_df_employee_shop, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_shop)
print('----------------------------------------------')
print(shap_df_employee_shop)
print('----------------------------------------------')
print(lime_df_employee_shop)

In [ ]:
df_final_svc_employee_shop

In [ ]:
breakdown_df_employee_smart, shap_df_employee_smart, lime_df_employee_smart, df_final_svc_employee_smart = preprocess_dfs(breakdown_df_employee_smart, 
                                                                                                                          shap_df_employee_smart, 
                                                                                                                          lime_df_employee_smart, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_smart)
print('----------------------------------------------')
print(shap_df_employee_smart)
print('----------------------------------------------')
print(lime_df_employee_smart)

In [ ]:
df_final_svc_employee_smart

### **Results Employee:**

In [ ]:
df_topfeatures_employee_comparch = top5_features(df_final_svc_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp = top5_features(df_final_svc_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest = top5_features(df_final_svc_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop = top5_features(df_final_svc_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart = top5_features(df_final_svc_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_employee = pd.concat([df_topfeatures_employee_comparch, df_topfeatures_employee_emp, df_topfeatures_employee_rest, 
                                     df_topfeatures_employee_shop, df_topfeatures_employee_smart])
df_topfeatures_employee

In [ ]:
df_topfeatures_employee_comparch_rank = top5_rank(df_final_svc_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp_rank = top5_rank(df_final_svc_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest_rank = top5_rank(df_final_svc_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop_rank = top5_rank(df_final_svc_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart_rank = top5_rank(df_final_svc_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_rank_employee = pd.concat([df_topfeatures_employee_comparch_rank, df_topfeatures_employee_emp_rank, 
                                          df_topfeatures_employee_rest_rank, df_topfeatures_employee_shop_rank, df_topfeatures_employee_smart_rank])
df_topfeatures_rank_employee

In [ ]:
df_topfeatures_employee_comparch_sign = top5_sign(df_final_svc_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp_sign = top5_sign(df_final_svc_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest_sign = top5_sign(df_final_svc_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop_sign = top5_sign(df_final_svc_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart_sign = top5_sign(df_final_svc_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_sign_employee = pd.concat([df_topfeatures_employee_comparch_sign, df_topfeatures_employee_emp_sign, 
                                          df_topfeatures_employee_rest_sign, df_topfeatures_employee_shop_sign, df_topfeatures_employee_smart_sign])
df_topfeatures_sign_employee

In [ ]:
df_topfeatures_employee_comparch_rank_sign = top5_rank_sign(df_final_svc_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp_rank_sign = top5_rank_sign(df_final_svc_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest_rank_sign = top5_rank_sign(df_final_svc_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop_rank_sign = top5_rank_sign(df_final_svc_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart_rank_sign = top5_rank_sign(df_final_svc_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_rank_sign_employee = pd.concat([df_topfeatures_employee_comparch_rank_sign, df_topfeatures_employee_emp_rank_sign, 
                                              df_topfeatures_employee_rest_rank_sign, df_topfeatures_employee_shop_rank_sign, 
                                              df_topfeatures_employee_smart_rank_sign])
df_topfeatures_rank_sign_employee

### **Restaurant:**

In [ ]:
breakdown_df_restaurant_comparch = svc_exp_results['breakdown_restaurant'][2].result
shap_df_restaurant_comparch = svc_exp_results['shap_restaurant'][2].result
lime_df_restaurant_comparch = svc_exp_results['lime_restaurant'][2].result

In [ ]:
breakdown_df_restaurant_emp = svc_exp_results['breakdown_restaurant'][4].result
shap_df_restaurant_emp = svc_exp_results['shap_restaurant'][4].result
lime_df_restaurant_emp = svc_exp_results['lime_restaurant'][4].result

In [ ]:
breakdown_df_restaurant_rest = svc_exp_results['breakdown_restaurant'][7].result
shap_df_restaurant_rest = svc_exp_results['shap_restaurant'][7].result
lime_df_restaurant_rest = svc_exp_results['lime_restaurant'][7].result

In [ ]:
breakdown_df_restaurant_shop = svc_exp_results['breakdown_restaurant'][8].result
shap_df_restaurant_shop = svc_exp_results['shap_restaurant'][8].result
lime_df_restaurant_shop = svc_exp_results['lime_restaurant'][8].result

In [ ]:
breakdown_df_restaurant_smart = svc_exp_results['breakdown_restaurant'][9].result
shap_df_restaurant_smart = svc_exp_results['shap_restaurant'][9].result
lime_df_restaurant_smart = svc_exp_results['lime_restaurant'][9].result

In [ ]:
exp_plots_bd = [svc_exp_results['breakdown_restaurant'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [svc_exp_results['shap_restaurant'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  svc_exp_results['lime_restaurant'][i].plot()

In [ ]:
breakdown_df_restaurant_comparch, shap_df_restaurant_comparch, lime_df_restaurant_comparch, df_final_svc_restaurant_comparch = preprocess_dfs(breakdown_df_restaurant_comparch, 
                                                                                                                                              shap_df_restaurant_comparch, 
                                                                                                                                              lime_df_restaurant_comparch, 
                                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_comparch)
print('----------------------------------------------')
print(shap_df_restaurant_comparch)
print('----------------------------------------------')
print(lime_df_restaurant_comparch)

In [ ]:
df_final_svc_restaurant_comparch

In [ ]:
breakdown_df_restaurant_emp, shap_df_restaurant_emp, lime_df_restaurant_emp, df_final_svc_restaurant_emp = preprocess_dfs(breakdown_df_restaurant_emp, 
                                                                                                                          shap_df_restaurant_emp, 
                                                                                                                          lime_df_restaurant_emp, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_emp)
print('----------------------------------------------')
print(shap_df_restaurant_emp)
print('----------------------------------------------')
print(lime_df_restaurant_emp)

In [ ]:
df_final_svc_restaurant_emp

In [ ]:
breakdown_df_restaurant_rest, shap_df_restaurant_rest, lime_df_restaurant_rest, df_final_svc_restaurant_rest = preprocess_dfs(breakdown_df_restaurant_rest, 
                                                                                                                              shap_df_restaurant_rest, 
                                                                                                                              lime_df_restaurant_rest, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_rest)
print('----------------------------------------------')
print(shap_df_restaurant_rest)
print('----------------------------------------------')
print(lime_df_restaurant_rest)

In [ ]:
df_final_svc_restaurant_rest

In [ ]:
breakdown_df_restaurant_shop, shap_df_restaurant_shop, lime_df_restaurant_shop, df_final_svc_restaurant_shop = preprocess_dfs(breakdown_df_restaurant_shop, 
                                                                                                                              shap_df_restaurant_shop, 
                                                                                                                              lime_df_restaurant_shop, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_shop)
print('----------------------------------------------')
print(shap_df_restaurant_shop)
print('----------------------------------------------')
print(lime_df_restaurant_shop)

In [ ]:
df_final_svc_restaurant_shop

In [ ]:
breakdown_df_restaurant_smart, shap_df_restaurant_smart, lime_df_restaurant_smart, df_final_svc_restaurant_smart = preprocess_dfs(breakdown_df_restaurant_smart, 
                                                                                                                                  shap_df_restaurant_smart, 
                                                                                                                                  lime_df_restaurant_smart, 
                                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_smart)
print('----------------------------------------------')
print(shap_df_restaurant_smart)
print('----------------------------------------------')
print(lime_df_restaurant_smart)

In [ ]:
df_final_svc_restaurant_smart

### **Results Restaurant:**

In [ ]:
df_topfeatures_restaurant_comparch = top5_features(df_final_svc_restaurant_comparch, 'Restaurant - Class: ComputerArchitecture')
df_topfeatures_restaurant_emp = top5_features(df_final_svc_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest = top5_features(df_final_svc_restaurant_rest, 'Restaurant- Class: Restaurant')
df_topfeatures_restaurant_shop = top5_features(df_final_svc_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart = top5_features(df_final_svc_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_restaurant = pd.concat([df_topfeatures_restaurant_comparch, df_topfeatures_restaurant_emp, df_topfeatures_restaurant_rest, 
                                       df_topfeatures_restaurant_shop, df_topfeatures_restaurant_smart])
df_topfeatures_restaurant

In [ ]:
df_topfeatures_restaurant_comparch_rank = top5_rank(df_final_svc_restaurant_comparch, 'Restaurant- Class: ComputerArchitecture')
df_topfeatures_restaurant_emp_rank = top5_rank(df_final_svc_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest_rank = top5_rank(df_final_svc_restaurant_rest, 'Restaurant - Class: Restaurant')
df_topfeatures_restaurant_shop_rank = top5_rank(df_final_svc_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart_rank = top5_rank(df_final_svc_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_rank_restaurant= pd.concat([df_topfeatures_restaurant_comparch_rank, df_topfeatures_restaurant_emp_rank, 
                                           df_topfeatures_restaurant_rest_rank, df_topfeatures_restaurant_shop_rank, df_topfeatures_restaurant_smart_rank])
df_topfeatures_rank_restaurant

In [ ]:
df_topfeatures_restaurant_comparch_sign = top5_sign(df_final_svc_restaurant_comparch, 'Restaurant - Class: ComputerArchitecture')
df_topfeatures_restaurant_emp_sign = top5_sign(df_final_svc_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest_sign = top5_sign(df_final_svc_restaurant_rest, 'Restaurant - Class: Restaurant')
df_topfeatures_restaurant_shop_sign = top5_sign(df_final_svc_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart_sign = top5_sign(df_final_svc_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_sign_restaurant = pd.concat([df_topfeatures_restaurant_comparch_sign, df_topfeatures_restaurant_emp_sign, 
                                            df_topfeatures_restaurant_rest_sign, df_topfeatures_restaurant_shop_sign, df_topfeatures_restaurant_smart_sign])
df_topfeatures_sign_restaurant

In [ ]:
df_topfeatures_restaurant_comparch_rank_sign = top5_rank_sign(df_final_svc_restaurant_comparch, 'Restaurant - Class: ComputerArchitecture')
df_topfeatures_restaurant_emp_rank_sign = top5_rank_sign(df_final_svc_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest_rank_sign = top5_rank_sign(df_final_svc_restaurant_rest, 'Restaurant - Class: Restaurant')
df_topfeatures_restaurant_shop_rank_sign = top5_rank_sign(df_final_svc_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart_rank_sign = top5_rank_sign(df_final_svc_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_rank_sign_restaurant = pd.concat([df_topfeatures_restaurant_comparch_rank_sign, df_topfeatures_restaurant_emp_rank_sign, 
                                                 df_topfeatures_restaurant_rest_rank_sign, df_topfeatures_restaurant_shop_rank_sign, 
                                                 df_topfeatures_restaurant_smart_rank_sign])
df_topfeatures_rank_sign_restaurant

### **Shopping:**

In [ ]:
breakdown_df_shopping_comparch = svc_exp_results['breakdown_shopping'][2].result
shap_df_shopping_comparch = svc_exp_results['shap_shopping'][2].result
lime_df_shopping_comparch = svc_exp_results['lime_shopping'][2].result

In [ ]:
breakdown_df_shopping_emp = svc_exp_results['breakdown_shopping'][4].result
shap_df_shopping_emp = svc_exp_results['shap_shopping'][4].result
lime_df_shopping_emp = svc_exp_results['lime_shopping'][4].result

In [ ]:
breakdown_df_shopping_rest = svc_exp_results['breakdown_shopping'][7].result
shap_df_shopping_rest = svc_exp_results['shap_shopping'][7].result
lime_df_shopping_rest = svc_exp_results['lime_shopping'][7].result

In [ ]:
breakdown_df_shopping_shop = svc_exp_results['breakdown_shopping'][8].result
shap_df_shopping_shop = svc_exp_results['shap_shopping'][8].result
lime_df_shopping_shop = svc_exp_results['lime_shopping'][8].result

In [ ]:
breakdown_df_shopping_smart = svc_exp_results['breakdown_shopping'][9].result
shap_df_shopping_smart = svc_exp_results['shap_shopping'][9].result
lime_df_shopping_smart = svc_exp_results['lime_shopping'][9].result

In [ ]:
exp_plots_bd = [svc_exp_results['breakdown_shopping'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [svc_exp_results['shap_shopping'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  svc_exp_results['lime_shopping'][i].plot()

In [ ]:
breakdown_df_shopping_comparch, shap_df_shopping_comparch, lime_df_shopping_comparch, df_final_svc_shopping_comparch = preprocess_dfs(breakdown_df_shopping_comparch, 
                                                                                                                                              shap_df_shopping_comparch, 
                                                                                                                                              lime_df_shopping_comparch, 
                                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_comparch)
print('----------------------------------------------')
print(shap_df_shopping_comparch)
print('----------------------------------------------')
print(lime_df_shopping_comparch)

In [ ]:
df_final_svc_shopping_comparch

In [ ]:
breakdown_df_shopping_emp, shap_df_shopping_emp, lime_df_shopping_emp, df_final_svc_shopping_emp = preprocess_dfs(breakdown_df_shopping_emp, 
                                                                                                                          shap_df_shopping_emp, 
                                                                                                                          lime_df_shopping_emp, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_emp)
print('----------------------------------------------')
print(shap_df_shopping_emp)
print('----------------------------------------------')
print(lime_df_shopping_emp)

In [ ]:
df_final_svc_shopping_emp

In [ ]:
breakdown_df_shopping_rest, shap_df_shopping_rest, lime_df_shopping_rest, df_final_svc_shopping_rest = preprocess_dfs(breakdown_df_shopping_rest, 
                                                                                                                              shap_df_shopping_rest, 
                                                                                                                              lime_df_shopping_rest, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_rest)
print('----------------------------------------------')
print(shap_df_shopping_rest)
print('----------------------------------------------')
print(lime_df_shopping_rest)

In [ ]:
df_final_svc_shopping_rest

In [ ]:
breakdown_df_shopping_shop, shap_df_shopping_shop, lime_df_shopping_shop, df_final_svc_shopping_shop = preprocess_dfs(breakdown_df_shopping_shop, 
                                                                                                                              shap_df_shopping_shop, 
                                                                                                                              lime_df_shopping_shop, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_shop)
print('----------------------------------------------')
print(shap_df_shopping_shop)
print('----------------------------------------------')
print(lime_df_shopping_shop)

In [ ]:
df_final_svc_shopping_shop

In [ ]:
breakdown_df_shopping_smart, shap_df_shopping_smart, lime_df_shopping_smart, df_final_svc_shopping_smart = preprocess_dfs(breakdown_df_shopping_smart, 
                                                                                                                                  shap_df_shopping_smart, 
                                                                                                                                  lime_df_shopping_smart, 
                                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_smart)
print('----------------------------------------------')
print(shap_df_shopping_smart)
print('----------------------------------------------')
print(lime_df_shopping_smart)

In [ ]:
df_final_svc_shopping_smart

### **Results Shopping:**

In [ ]:
df_topfeatures_shopping_comparch = top5_features(df_final_svc_shopping_comparch, 'Shopping - Class: ComputerArchitecture')
df_topfeatures_shopping_emp = top5_features(df_final_svc_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest = top5_features(df_final_svc_shopping_rest, 'Shopping - Class: Restaurant')
df_topfeatures_shopping_shop = top5_features(df_final_svc_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart = top5_features(df_final_svc_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_shopping = pd.concat([df_topfeatures_shopping_comparch, df_topfeatures_shopping_emp, df_topfeatures_shopping_rest, 
                                       df_topfeatures_shopping_shop, df_topfeatures_shopping_smart])
df_topfeatures_shopping

In [ ]:
df_topfeatures_shopping_comparch_rank = top5_rank(df_final_svc_shopping_comparch, 'Shopping - Class: ComputerArchitecture')
df_topfeatures_shopping_emp_rank = top5_rank(df_final_svc_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest_rank = top5_rank(df_final_svc_shopping_rest, 'Shopping  - Class: Restaurant')
df_topfeatures_shopping_shop_rank = top5_rank(df_final_svc_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart_rank = top5_rank(df_final_svc_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_rank_shopping= pd.concat([df_topfeatures_shopping_comparch_rank, df_topfeatures_shopping_emp_rank, 
                                           df_topfeatures_shopping_rest_rank, df_topfeatures_shopping_shop_rank, df_topfeatures_shopping_smart_rank])
df_topfeatures_rank_shopping

In [ ]:
df_topfeatures_shopping_comparch_sign = top5_sign(df_final_svc_shopping_comparch, 'Shopping  - Class: ComputerArchitecture')
df_topfeatures_shopping_emp_sign = top5_sign(df_final_svc_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest_sign = top5_sign(df_final_svc_shopping_rest, 'Shopping  - Class: Restaurant')
df_topfeatures_shopping_shop_sign = top5_sign(df_final_svc_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart_sign = top5_sign(df_final_svc_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_sign_shopping = pd.concat([df_topfeatures_shopping_comparch_sign, df_topfeatures_shopping_emp_sign, 
                                            df_topfeatures_shopping_rest_sign, df_topfeatures_shopping_shop_sign, df_topfeatures_shopping_smart_sign])
df_topfeatures_sign_shopping

In [ ]:
df_topfeatures_shopping_comparch_rank_sign = top5_rank_sign(df_final_svc_shopping_comparch, 'Shopping  - Class: ComputerArchitecture')
df_topfeatures_shopping_emp_rank_sign = top5_rank_sign(df_final_svc_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest_rank_sign = top5_rank_sign(df_final_svc_shopping_rest, 'Shopping  - Class: Restaurant')
df_topfeatures_shopping_shop_rank_sign = top5_rank_sign(df_final_svc_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart_rank_sign = top5_rank_sign(df_final_svc_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_rank_sign_shopping = pd.concat([df_topfeatures_shopping_comparch_rank_sign, df_topfeatures_shopping_emp_rank_sign, 
                                                 df_topfeatures_shopping_rest_rank_sign, df_topfeatures_shopping_shop_rank_sign, 
                                                 df_topfeatures_shopping_smart_rank_sign])
df_topfeatures_rank_sign_shopping

### **SmartHouse:**

In [ ]:
breakdown_df_smarthouse_comparch = svc_exp_results['breakdown_smarthouse'][2].result
shap_df_smarthouse_comparch = svc_exp_results['shap_smarthouse'][2].result
lime_df_smarthouse_comparch = svc_exp_results['lime_smarthouse'][2].result

In [ ]:
breakdown_df_smarthouse_emp = svc_exp_results['breakdown_smarthouse'][4].result
shap_df_smarthouse_emp = svc_exp_results['shap_smarthouse'][4].result
lime_df_smarthouse_emp = svc_exp_results['lime_smarthouse'][4].result

In [ ]:
breakdown_df_smarthouse_rest = svc_exp_results['breakdown_smarthouse'][7].result
shap_df_smarthouse_rest = svc_exp_results['shap_smarthouse'][7].result
lime_df_smarthouse_rest = svc_exp_results['lime_smarthouse'][7].result

In [ ]:
breakdown_df_smarthouse_shop = svc_exp_results['breakdown_smarthouse'][8].result
shap_df_smarthouse_shop = svc_exp_results['shap_smarthouse'][8].result
lime_df_smarthouse_shop = svc_exp_results['lime_smarthouse'][8].result

In [ ]:
breakdown_df_smarthouse_smart = svc_exp_results['breakdown_smarthouse'][9].result
shap_df_smarthouse_smart = svc_exp_results['shap_smarthouse'][9].result
lime_df_smarthouse_smart = svc_exp_results['lime_smarthouse'][9].result

In [ ]:
exp_plots_bd = [svc_exp_results['breakdown_smarthouse'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [svc_exp_results['shap_smarthouse'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  svc_exp_results['lime_smarthouse'][i].plot()

In [ ]:
breakdown_df_smarthouse_comparch, shap_df_smarthouse_comparch, lime_df_smarthouse_comparch, df_final_svc_smarthouse_comparch = preprocess_dfs(breakdown_df_smarthouse_comparch, 
                                                                                                                                      shap_df_smarthouse_comparch, 
                                                                                                                                      lime_df_smarthouse_comparch, 
                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_comparch)
print('----------------------------------------------')
print(shap_df_smarthouse_comparch)
print('----------------------------------------------')
print(lime_df_smarthouse_comparch)

In [ ]:
df_final_svc_smarthouse_comparch

In [ ]:
breakdown_df_smarthouse_emp, shap_df_smarthouse_emp, lime_df_smarthouse_emp, df_final_svc_smarthouse_emp = preprocess_dfs(breakdown_df_smarthouse_emp, 
                                                                                                                  shap_df_smarthouse_emp, 
                                                                                                                  lime_df_smarthouse_emp, 
                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_emp)
print('----------------------------------------------')
print(shap_df_smarthouse_emp)
print('----------------------------------------------')
print(lime_df_smarthouse_emp)

In [ ]:
df_final_svc_smarthouse_emp

In [ ]:
breakdown_df_smarthouse_rest, shap_df_smarthouse_rest, lime_df_smarthouse_rest, df_final_svc_smarthouse_rest = preprocess_dfs(breakdown_df_smarthouse_rest, 
                                                                                                                      shap_df_smarthouse_rest, 
                                                                                                                      lime_df_smarthouse_rest, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_rest)
print('----------------------------------------------')
print(shap_df_smarthouse_rest)
print('----------------------------------------------')
print(lime_df_smarthouse_rest)

In [ ]:
df_final_svc_smarthouse_rest

In [ ]:
breakdown_df_smarthouse_shop, shap_df_smarthouse_shop, lime_df_smarthouse_shop, df_final_svc_smarthouse_shop = preprocess_dfs(breakdown_df_smarthouse_shop, 
                                                                                                                      shap_df_smarthouse_shop, 
                                                                                                                      lime_df_smarthouse_shop, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_shop)
print('----------------------------------------------')
print(shap_df_smarthouse_shop)
print('----------------------------------------------')
print(lime_df_smarthouse_shop)

In [ ]:
df_final_svc_smarthouse_shop

In [ ]:
breakdown_df_smarthouse_smart, shap_df_smarthouse_smart, lime_df_smarthouse_smart, df_final_svc_smarthouse_smart = preprocess_dfs(breakdown_df_smarthouse_smart, 
                                                                                                                          shap_df_smarthouse_smart, 
                                                                                                                          lime_df_smarthouse_smart, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_smart)
print('----------------------------------------------')
print(shap_df_smarthouse_smart)
print('----------------------------------------------')
print(lime_df_smarthouse_smart)

In [ ]:
df_final_svc_smarthouse_smart

### **Results SmartHouse:**

In [ ]:
df_topfeatures_smarthouse_comparch = top5_features(df_final_svc_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp = top5_features(df_final_svc_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest = top5_features(df_final_svc_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop = top5_features(df_final_svc_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart = top5_features(df_final_svc_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch, df_topfeatures_smarthouse_emp, df_topfeatures_smarthouse_rest, 
                                     df_topfeatures_smarthouse_shop, df_topfeatures_smarthouse_smart])
df_topfeatures_smarthouse

In [ ]:
df_topfeatures_smarthouse_comparch_rank = top5_rank(df_final_svc_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp_rank = top5_rank(df_final_svc_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest_rank = top5_rank(df_final_svc_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop_rank = top5_rank(df_final_svc_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart_rank = top5_rank(df_final_svc_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_rank_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch_rank, df_topfeatures_smarthouse_emp_rank, 
                                          df_topfeatures_smarthouse_rest_rank, df_topfeatures_smarthouse_shop_rank, df_topfeatures_smarthouse_smart_rank])
df_topfeatures_rank_smarthouse

In [ ]:
df_topfeatures_smarthouse_comparch_sign = top5_sign(df_final_svc_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp_sign = top5_sign(df_final_svc_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest_sign = top5_sign(df_final_svc_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop_sign = top5_sign(df_final_svc_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart_sign = top5_sign(df_final_svc_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_sign_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch_sign, df_topfeatures_smarthouse_emp_sign, 
                                          df_topfeatures_smarthouse_rest_sign, df_topfeatures_smarthouse_shop_sign, df_topfeatures_smarthouse_smart_sign])
df_topfeatures_sign_smarthouse

In [ ]:
df_topfeatures_smarthouse_comparch_rank_sign = top5_rank_sign(df_final_svc_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp_rank_sign = top5_rank_sign(df_final_svc_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest_rank_sign = top5_rank_sign(df_final_svc_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop_rank_sign = top5_rank_sign(df_final_svc_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart_rank_sign = top5_rank_sign(df_final_svc_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_rank_sign_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch_rank_sign, df_topfeatures_smarthouse_emp_rank_sign, 
                                              df_topfeatures_smarthouse_rest_rank_sign, df_topfeatures_smarthouse_shop_rank_sign, 
                                              df_topfeatures_smarthouse_smart_rank_sign])
df_topfeatures_rank_sign_smarthouse

## RF:

In [ ]:
exp_list_rf = []

for i in range(len(unique_values)):
    class_value = unique_values[i]
    pf = lambda m, d, i=i: m.predict_proba(d)[:, i]
    e = dx.Explainer(
        model_rf, X_train,
        y_train,
        predict_function=pf,
        label=f'RFClassifier: class {class_value}',
        verbose=False
    )
    exp_list_rf += [e]

exp_list_rf

In [ ]:
inst_computerarchitecture = X_test.loc[inst_comp_rf]
inst_restaurant = X_test.loc[inst_rest_rf]
inst_shopping = X_test.loc[inst_shop_rf]
inst_employee = X_test.loc[inst_emp_rf]
inst_smarthouse = X_test.loc[inst_smart_rf]

In [ ]:
rf_exp_results = {}

for instance in inst_names:
    breakdown_exp = [exp.predict_parts(eval(f"inst_{instance}"), type='break_down', random_state=42) for exp in exp_list_rf]
    rf_exp_results[f"breakdown_{instance}"] = breakdown_exp

    shap_exp = [exp.predict_parts(eval(f"inst_{instance}"), type='shap', random_state=42) for exp in exp_list_rf]
    rf_exp_results[f"shap_{instance}"] = shap_exp

    lime_exp = [exp.predict_surrogate(eval(f"inst_{instance}"), random_state=42, class_names=[0, unique_values[i]]) for i, exp in enumerate(exp_list_rf)]
    rf_exp_results[f"lime_{instance}"] = lime_exp

### **Computer-architecture:**

In [ ]:
breakdown_df_computerarchitecture_comparch = rf_exp_results['breakdown_computerarchitecture'][2].result
shap_df_computerarchitecture_comparch = rf_exp_results['shap_computerarchitecture'][2].result
lime_df_computerarchitecture_comparch = rf_exp_results['lime_computerarchitecture'][2].result

In [ ]:
breakdown_df_computerarchitecture_emp = rf_exp_results['breakdown_computerarchitecture'][4].result
shap_df_computerarchitecture_emp = rf_exp_results['shap_computerarchitecture'][4].result
lime_df_computerarchitecture_emp = rf_exp_results['lime_computerarchitecture'][4].result

In [ ]:
breakdown_df_computerarchitecture_rest = rf_exp_results['breakdown_computerarchitecture'][7].result
shap_df_computerarchitecture_rest = rf_exp_results['shap_computerarchitecture'][7].result
lime_df_computerarchitecture_rest = rf_exp_results['lime_computerarchitecture'][7].result

In [ ]:
breakdown_df_computerarchitecture_shop = rf_exp_results['breakdown_computerarchitecture'][8].result
shap_df_computerarchitecture_shop = rf_exp_results['shap_computerarchitecture'][8].result
lime_df_computerarchitecture_shop = rf_exp_results['lime_computerarchitecture'][8].result

In [ ]:
breakdown_df_computerarchitecture_smart = rf_exp_results['breakdown_computerarchitecture'][9].result
shap_df_computerarchitecture_smart = rf_exp_results['shap_computerarchitecture'][9].result
lime_df_computerarchitecture_smart = rf_exp_results['lime_computerarchitecture'][9].result

In [ ]:
exp_plots_bd = [rf_exp_results['breakdown_computerarchitecture'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [rf_exp_results['shap_computerarchitecture'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  rf_exp_results['lime_computerarchitecture'][i].plot()

In [ ]:
breakdown_df_computerarchitecture_comparch, shap_df_computerarchitecture_comparch, lime_df_computerarchitecture_comparch, df_final_rf_computerarchitecture_comparch = preprocess_dfs(breakdown_df_computerarchitecture_comparch, 
                                                                                                                                                                                      shap_df_computerarchitecture_comparch, 
                                                                                                                                                                                      lime_df_computerarchitecture_comparch, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_comparch)
print('----------------------------------------------')
print(shap_df_computerarchitecture_comparch)
print('----------------------------------------------')
print(lime_df_computerarchitecture_comparch)

In [ ]:
df_final_rf_computerarchitecture_comparch

In [ ]:
breakdown_df_computerarchitecture_emp, shap_df_computerarchitecture_emp, lime_df_computerarchitecture_emp, df_final_rf_computerarchitecture_emp = preprocess_dfs(breakdown_df_computerarchitecture_emp, 
                                                                                                                                                                                      shap_df_computerarchitecture_emp, 
                                                                                                                                                                                      lime_df_computerarchitecture_emp, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_emp)
print('----------------------------------------------')
print(shap_df_computerarchitecture_emp)
print('----------------------------------------------')
print(lime_df_computerarchitecture_emp)

In [ ]:
df_final_rf_computerarchitecture_emp

In [ ]:
breakdown_df_computerarchitecture_rest, shap_df_computerarchitecture_rest, lime_df_computerarchitecture_rest, df_final_rf_computerarchitecture_rest = preprocess_dfs(breakdown_df_computerarchitecture_rest, 
                                                                                                                                                                                      shap_df_computerarchitecture_rest, 
                                                                                                                                                                                      lime_df_computerarchitecture_rest, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_rest)
print('----------------------------------------------')
print(shap_df_computerarchitecture_rest)
print('----------------------------------------------')
print(lime_df_computerarchitecture_rest)

In [ ]:
df_final_rf_computerarchitecture_rest

In [ ]:
breakdown_df_computerarchitecture_shop, shap_df_computerarchitecture_shop, lime_df_computerarchitecture_shop, df_final_rf_computerarchitecture_shop = preprocess_dfs(breakdown_df_computerarchitecture_shop, 
                                                                                                                                                                                      shap_df_computerarchitecture_shop, 
                                                                                                                                                                                      lime_df_computerarchitecture_shop, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_shop)
print('----------------------------------------------')
print(shap_df_computerarchitecture_shop)
print('----------------------------------------------')
print(lime_df_computerarchitecture_shop)

In [ ]:
df_final_rf_computerarchitecture_shop

In [ ]:
breakdown_df_computerarchitecture_smart, shap_df_computerarchitecture_smart, lime_df_computerarchitecture_smart, df_final_rf_computerarchitecture_smart = preprocess_dfs(breakdown_df_computerarchitecture_smart, 
                                                                                                                                                                                      shap_df_computerarchitecture_smart, 
                                                                                                                                                                                      lime_df_computerarchitecture_smart, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_smart)
print('----------------------------------------------')
print(shap_df_computerarchitecture_smart)
print('----------------------------------------------')
print(lime_df_computerarchitecture_smart)

In [ ]:
df_final_rf_computerarchitecture_smart

### **Results Computer-architecture:**

In [ ]:
df_topfeatures_comparch_comparch = top5_features(df_final_rf_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp = top5_features(df_final_rf_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest = top5_features(df_final_rf_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop = top5_features(df_final_rf_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart = top5_features(df_final_rf_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch, df_topfeatures_comparch_emp, df_topfeatures_comparch_rest, 
                                                 df_topfeatures_comparch_shop, df_topfeatures_comparch_smart])
df_topfeatures_computerarchitecture

In [ ]:
df_topfeatures_comparch_comparch_rank = top5_rank(df_final_rf_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp_rank = top5_rank(df_final_rf_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest_rank = top5_rank(df_final_rf_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop_rank = top5_rank(df_final_rf_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart_rank = top5_rank(df_final_rf_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_rank_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch_rank, df_topfeatures_comparch_emp_rank, 
                                                      df_topfeatures_comparch_rest_rank, df_topfeatures_comparch_shop_rank, df_topfeatures_comparch_smart_rank])
df_topfeatures_rank_computerarchitecture

In [ ]:
df_topfeatures_comparch_comparch_sign = top5_sign(df_final_rf_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp_sign = top5_sign(df_final_rf_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest_sign = top5_sign(df_final_rf_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop_sign = top5_sign(df_final_rf_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart_sign = top5_sign(df_final_rf_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_sign_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch_sign, df_topfeatures_comparch_emp_sign, 
                                                      df_topfeatures_comparch_rest_sign, df_topfeatures_comparch_shop_sign, df_topfeatures_comparch_smart_sign])
df_topfeatures_sign_computerarchitecture

In [ ]:
df_topfeatures_comparch_comparch_rank_sign = top5_rank_sign(df_final_rf_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp_rank_sign = top5_rank_sign(df_final_rf_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest_rank_sign = top5_rank_sign(df_final_rf_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop_rank_sign = top5_rank_sign(df_final_rf_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart_rank_sign = top5_rank_sign(df_final_rf_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_rank_sign_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch_rank_sign, df_topfeatures_comparch_emp_rank_sign, 
                                                           df_topfeatures_comparch_rest_rank_sign, df_topfeatures_comparch_shop_rank_sign, 
                                                           df_topfeatures_comparch_smart_rank_sign])
df_topfeatures_rank_sign_computerarchitecture

### **Employee:**

In [ ]:
breakdown_df_employee_comparch = rf_exp_results['breakdown_employee'][2].result
shap_df_employee_comparch = rf_exp_results['shap_employee'][2].result
lime_df_employee_comparch = rf_exp_results['lime_employee'][2].result

In [ ]:
breakdown_df_employee_emp = rf_exp_results['breakdown_employee'][4].result
shap_df_employee_emp = rf_exp_results['shap_employee'][4].result
lime_df_employee_emp = rf_exp_results['lime_employee'][4].result

In [ ]:
breakdown_df_employee_rest = rf_exp_results['breakdown_employee'][7].result
shap_df_employee_rest = rf_exp_results['shap_employee'][7].result
lime_df_employee_rest = rf_exp_results['lime_employee'][7].result

In [ ]:
breakdown_df_employee_shop = rf_exp_results['breakdown_employee'][8].result
shap_df_employee_shop = rf_exp_results['shap_employee'][8].result
lime_df_employee_shop = rf_exp_results['lime_employee'][8].result

In [ ]:
breakdown_df_employee_smart = rf_exp_results['breakdown_employee'][9].result
shap_df_employee_smart = rf_exp_results['shap_employee'][9].result
lime_df_employee_smart = rf_exp_results['lime_employee'][9].result

In [ ]:
exp_plots_bd = [rf_exp_results['breakdown_employee'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [rf_exp_results['shap_employee'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  rf_exp_results['lime_employee'][i].plot()

In [ ]:
breakdown_df_employee_comparch, shap_df_employee_comparch, lime_df_employee_comparch, df_final_rf_employee_comparch = preprocess_dfs(breakdown_df_employee_comparch, 
                                                                                                                                      shap_df_employee_comparch, 
                                                                                                                                      lime_df_employee_comparch, 
                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_comparch)
print('----------------------------------------------')
print(shap_df_employee_comparch)
print('----------------------------------------------')
print(lime_df_employee_comparch)

In [ ]:
df_final_rf_employee_comparch

In [ ]:
breakdown_df_employee_emp, shap_df_employee_emp, lime_df_employee_emp, df_final_rf_employee_emp = preprocess_dfs(breakdown_df_employee_emp, 
                                                                                                                  shap_df_employee_emp, 
                                                                                                                  lime_df_employee_emp, 
                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_emp)
print('----------------------------------------------')
print(shap_df_employee_emp)
print('----------------------------------------------')
print(lime_df_employee_emp)

In [ ]:
df_final_rf_employee_emp

In [ ]:
breakdown_df_employee_rest, shap_df_employee_rest, lime_df_employee_rest, df_final_rf_employee_rest = preprocess_dfs(breakdown_df_employee_rest, 
                                                                                                                      shap_df_employee_rest, 
                                                                                                                      lime_df_employee_rest, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_rest)
print('----------------------------------------------')
print(shap_df_employee_rest)
print('----------------------------------------------')
print(lime_df_employee_rest)

In [ ]:
df_final_rf_employee_rest

In [ ]:
breakdown_df_employee_shop, shap_df_employee_shop, lime_df_employee_shop, df_final_rf_employee_shop = preprocess_dfs(breakdown_df_employee_shop, 
                                                                                                                      shap_df_employee_shop, 
                                                                                                                      lime_df_employee_shop, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_shop)
print('----------------------------------------------')
print(shap_df_employee_shop)
print('----------------------------------------------')
print(lime_df_employee_shop)

In [ ]:
df_final_rf_employee_shop

In [ ]:
breakdown_df_employee_smart, shap_df_employee_smart, lime_df_employee_smart, df_final_rf_employee_smart = preprocess_dfs(breakdown_df_employee_smart, 
                                                                                                                          shap_df_employee_smart, 
                                                                                                                          lime_df_employee_smart, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_smart)
print('----------------------------------------------')
print(shap_df_employee_smart)
print('----------------------------------------------')
print(lime_df_employee_smart)

In [ ]:
df_final_rf_employee_smart

### **Results Employee:**

In [ ]:
df_topfeatures_employee_comparch = top5_features(df_final_rf_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp = top5_features(df_final_rf_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest = top5_features(df_final_rf_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop = top5_features(df_final_rf_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart = top5_features(df_final_rf_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_employee = pd.concat([df_topfeatures_employee_comparch, df_topfeatures_employee_emp, df_topfeatures_employee_rest, 
                                     df_topfeatures_employee_shop, df_topfeatures_employee_smart])
df_topfeatures_employee

In [ ]:
df_topfeatures_employee_comparch_rank = top5_rank(df_final_rf_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp_rank = top5_rank(df_final_rf_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest_rank = top5_rank(df_final_rf_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop_rank = top5_rank(df_final_rf_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart_rank = top5_rank(df_final_rf_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_rank_employee = pd.concat([df_topfeatures_employee_comparch_rank, df_topfeatures_employee_emp_rank, 
                                          df_topfeatures_employee_rest_rank, df_topfeatures_employee_shop_rank, df_topfeatures_employee_smart_rank])
df_topfeatures_rank_employee

In [ ]:
df_topfeatures_employee_comparch_sign = top5_sign(df_final_rf_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp_sign = top5_sign(df_final_rf_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest_sign = top5_sign(df_final_rf_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop_sign = top5_sign(df_final_rf_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart_sign = top5_sign(df_final_rf_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_sign_employee = pd.concat([df_topfeatures_employee_comparch_sign, df_topfeatures_employee_emp_sign, 
                                          df_topfeatures_employee_rest_sign, df_topfeatures_employee_shop_sign, df_topfeatures_employee_smart_sign])
df_topfeatures_sign_employee

In [ ]:
df_topfeatures_employee_comparch_rank_sign = top5_rank_sign(df_final_rf_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp_rank_sign = top5_rank_sign(df_final_rf_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest_rank_sign = top5_rank_sign(df_final_rf_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop_rank_sign = top5_rank_sign(df_final_rf_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart_rank_sign = top5_rank_sign(df_final_rf_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_rank_sign_employee = pd.concat([df_topfeatures_employee_comparch_rank_sign, df_topfeatures_employee_emp_rank_sign, 
                                              df_topfeatures_employee_rest_rank_sign, df_topfeatures_employee_shop_rank_sign, 
                                              df_topfeatures_employee_smart_rank_sign])
df_topfeatures_rank_sign_employee

### **Restaurant:**

In [ ]:
breakdown_df_restaurant_comparch = rf_exp_results['breakdown_restaurant'][2].result
shap_df_restaurant_comparch = rf_exp_results['shap_restaurant'][2].result
lime_df_restaurant_comparch = rf_exp_results['lime_restaurant'][2].result

In [ ]:
breakdown_df_restaurant_emp = rf_exp_results['breakdown_restaurant'][4].result
shap_df_restaurant_emp = rf_exp_results['shap_restaurant'][4].result
lime_df_restaurant_emp = rf_exp_results['lime_restaurant'][4].result

In [ ]:
breakdown_df_restaurant_rest = rf_exp_results['breakdown_restaurant'][7].result
shap_df_restaurant_rest = rf_exp_results['shap_restaurant'][7].result
lime_df_restaurant_rest = rf_exp_results['lime_restaurant'][7].result

In [ ]:
breakdown_df_restaurant_shop = rf_exp_results['breakdown_restaurant'][8].result
shap_df_restaurant_shop = rf_exp_results['shap_restaurant'][8].result
lime_df_restaurant_shop = rf_exp_results['lime_restaurant'][8].result

In [ ]:
breakdown_df_restaurant_smart = rf_exp_results['breakdown_restaurant'][9].result
shap_df_restaurant_smart = rf_exp_results['shap_restaurant'][9].result
lime_df_restaurant_smart = rf_exp_results['lime_restaurant'][9].result

In [ ]:
exp_plots_bd = [rf_exp_results['breakdown_restaurant'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [rf_exp_results['shap_restaurant'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  rf_exp_results['lime_restaurant'][i].plot()

In [ ]:
breakdown_df_restaurant_comparch, shap_df_restaurant_comparch, lime_df_restaurant_comparch, df_final_rf_restaurant_comparch = preprocess_dfs(breakdown_df_restaurant_comparch, 
                                                                                                                                              shap_df_restaurant_comparch, 
                                                                                                                                              lime_df_restaurant_comparch, 
                                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_comparch)
print('----------------------------------------------')
print(shap_df_restaurant_comparch)
print('----------------------------------------------')
print(lime_df_restaurant_comparch)

In [ ]:
df_final_rf_restaurant_comparch

In [ ]:
breakdown_df_restaurant_emp, shap_df_restaurant_emp, lime_df_restaurant_emp, df_final_rf_restaurant_emp = preprocess_dfs(breakdown_df_restaurant_emp, 
                                                                                                                          shap_df_restaurant_emp, 
                                                                                                                          lime_df_restaurant_emp, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_emp)
print('----------------------------------------------')
print(shap_df_restaurant_emp)
print('----------------------------------------------')
print(lime_df_restaurant_emp)

In [ ]:
df_final_rf_restaurant_emp

In [ ]:
breakdown_df_restaurant_rest, shap_df_restaurant_rest, lime_df_restaurant_rest, df_final_rf_restaurant_rest = preprocess_dfs(breakdown_df_restaurant_rest, 
                                                                                                                              shap_df_restaurant_rest, 
                                                                                                                              lime_df_restaurant_rest, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_rest)
print('----------------------------------------------')
print(shap_df_restaurant_rest)
print('----------------------------------------------')
print(lime_df_restaurant_rest)

In [ ]:
df_final_rf_restaurant_rest

In [ ]:
breakdown_df_restaurant_shop, shap_df_restaurant_shop, lime_df_restaurant_shop, df_final_rf_restaurant_shop = preprocess_dfs(breakdown_df_restaurant_shop, 
                                                                                                                              shap_df_restaurant_shop, 
                                                                                                                              lime_df_restaurant_shop, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_shop)
print('----------------------------------------------')
print(shap_df_restaurant_shop)
print('----------------------------------------------')
print(lime_df_restaurant_shop)

In [ ]:
df_final_rf_restaurant_shop

In [ ]:
breakdown_df_restaurant_smart, shap_df_restaurant_smart, lime_df_restaurant_smart, df_final_rf_restaurant_smart = preprocess_dfs(breakdown_df_restaurant_smart, 
                                                                                                                                  shap_df_restaurant_smart, 
                                                                                                                                  lime_df_restaurant_smart, 
                                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_smart)
print('----------------------------------------------')
print(shap_df_restaurant_smart)
print('----------------------------------------------')
print(lime_df_restaurant_smart)

In [ ]:
df_final_rf_restaurant_smart

### **Results Restaurant:**

In [ ]:
df_topfeatures_restaurant_comparch = top5_features(df_final_rf_restaurant_comparch, 'Restaurant - Class: ComputerArchitecture')
df_topfeatures_restaurant_emp = top5_features(df_final_rf_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest = top5_features(df_final_rf_restaurant_rest, 'Restaurant- Class: Restaurant')
df_topfeatures_restaurant_shop = top5_features(df_final_rf_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart = top5_features(df_final_rf_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_restaurant = pd.concat([df_topfeatures_restaurant_comparch, df_topfeatures_restaurant_emp, df_topfeatures_restaurant_rest, 
                                       df_topfeatures_restaurant_shop, df_topfeatures_restaurant_smart])
df_topfeatures_restaurant

In [ ]:
df_topfeatures_restaurant_comparch_rank = top5_rank(df_final_rf_restaurant_comparch, 'Restaurant- Class: ComputerArchitecture')
df_topfeatures_restaurant_emp_rank = top5_rank(df_final_rf_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest_rank = top5_rank(df_final_rf_restaurant_rest, 'Restaurant - Class: Restaurant')
df_topfeatures_restaurant_shop_rank = top5_rank(df_final_rf_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart_rank = top5_rank(df_final_rf_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_rank_restaurant= pd.concat([df_topfeatures_restaurant_comparch_rank, df_topfeatures_restaurant_emp_rank, 
                                           df_topfeatures_restaurant_rest_rank, df_topfeatures_restaurant_shop_rank, df_topfeatures_restaurant_smart_rank])
df_topfeatures_rank_restaurant

In [ ]:
df_topfeatures_restaurant_comparch_sign = top5_sign(df_final_rf_restaurant_comparch, 'Restaurant - Class: ComputerArchitecture')
df_topfeatures_restaurant_emp_sign = top5_sign(df_final_rf_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest_sign = top5_sign(df_final_rf_restaurant_rest, 'Restaurant - Class: Restaurant')
df_topfeatures_restaurant_shop_sign = top5_sign(df_final_rf_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart_sign = top5_sign(df_final_rf_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_sign_restaurant = pd.concat([df_topfeatures_restaurant_comparch_sign, df_topfeatures_restaurant_emp_sign, 
                                            df_topfeatures_restaurant_rest_sign, df_topfeatures_restaurant_shop_sign, df_topfeatures_restaurant_smart_sign])
df_topfeatures_sign_restaurant

In [ ]:
df_topfeatures_restaurant_comparch_rank_sign = top5_rank_sign(df_final_rf_restaurant_comparch, 'Restaurant - Class: ComputerArchitecture')
df_topfeatures_restaurant_emp_rank_sign = top5_rank_sign(df_final_rf_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest_rank_sign = top5_rank_sign(df_final_rf_restaurant_rest, 'Restaurant - Class: Restaurant')
df_topfeatures_restaurant_shop_rank_sign = top5_rank_sign(df_final_rf_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart_rank_sign = top5_rank_sign(df_final_rf_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_rank_sign_restaurant = pd.concat([df_topfeatures_restaurant_comparch_rank_sign, df_topfeatures_restaurant_emp_rank_sign, 
                                                 df_topfeatures_restaurant_rest_rank_sign, df_topfeatures_restaurant_shop_rank_sign, 
                                                 df_topfeatures_restaurant_smart_rank_sign])
df_topfeatures_rank_sign_restaurant

### **Shopping:**

In [ ]:
breakdown_df_shopping_comparch = rf_exp_results['breakdown_shopping'][2].result
shap_df_shopping_comparch = rf_exp_results['shap_shopping'][2].result
lime_df_shopping_comparch = rf_exp_results['lime_shopping'][2].result

In [ ]:
breakdown_df_shopping_emp = rf_exp_results['breakdown_shopping'][4].result
shap_df_shopping_emp = rf_exp_results['shap_shopping'][4].result
lime_df_shopping_emp = rf_exp_results['lime_shopping'][4].result

In [ ]:
breakdown_df_shopping_rest = rf_exp_results['breakdown_shopping'][7].result
shap_df_shopping_rest = rf_exp_results['shap_shopping'][7].result
lime_df_shopping_rest = rf_exp_results['lime_shopping'][7].result

In [ ]:
breakdown_df_shopping_shop = rf_exp_results['breakdown_shopping'][8].result
shap_df_shopping_shop = rf_exp_results['shap_shopping'][8].result
lime_df_shopping_shop = rf_exp_results['lime_shopping'][8].result

In [ ]:
breakdown_df_shopping_smart = rf_exp_results['breakdown_shopping'][9].result
shap_df_shopping_smart = rf_exp_results['shap_shopping'][9].result
lime_df_shopping_smart = rf_exp_results['lime_shopping'][9].result

In [ ]:
exp_plots_bd = [rf_exp_results['breakdown_shopping'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [rf_exp_results['shap_shopping'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  rf_exp_results['lime_shopping'][i].plot()

In [ ]:
breakdown_df_shopping_comparch, shap_df_shopping_comparch, lime_df_shopping_comparch, df_final_rf_shopping_comparch = preprocess_dfs(breakdown_df_shopping_comparch, 
                                                                                                                                              shap_df_shopping_comparch, 
                                                                                                                                              lime_df_shopping_comparch, 
                                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_comparch)
print('----------------------------------------------')
print(shap_df_shopping_comparch)
print('----------------------------------------------')
print(lime_df_shopping_comparch)

In [ ]:
df_final_rf_shopping_comparch

In [ ]:
breakdown_df_shopping_emp, shap_df_shopping_emp, lime_df_shopping_emp, df_final_rf_shopping_emp = preprocess_dfs(breakdown_df_shopping_emp, 
                                                                                                                          shap_df_shopping_emp, 
                                                                                                                          lime_df_shopping_emp, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_emp)
print('----------------------------------------------')
print(shap_df_shopping_emp)
print('----------------------------------------------')
print(lime_df_shopping_emp)

In [ ]:
df_final_rf_shopping_emp

In [ ]:
breakdown_df_shopping_rest, shap_df_shopping_rest, lime_df_shopping_rest, df_final_rf_shopping_rest = preprocess_dfs(breakdown_df_shopping_rest, 
                                                                                                                              shap_df_shopping_rest, 
                                                                                                                              lime_df_shopping_rest, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_rest)
print('----------------------------------------------')
print(shap_df_shopping_rest)
print('----------------------------------------------')
print(lime_df_shopping_rest)

In [ ]:
df_final_rf_shopping_rest

In [ ]:
breakdown_df_shopping_shop, shap_df_shopping_shop, lime_df_shopping_shop, df_final_rf_shopping_shop = preprocess_dfs(breakdown_df_shopping_shop, 
                                                                                                                              shap_df_shopping_shop, 
                                                                                                                              lime_df_shopping_shop, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_shop)
print('----------------------------------------------')
print(shap_df_shopping_shop)
print('----------------------------------------------')
print(lime_df_shopping_shop)

In [ ]:
df_final_rf_shopping_shop

In [ ]:
breakdown_df_shopping_smart, shap_df_shopping_smart, lime_df_shopping_smart, df_final_rf_shopping_smart = preprocess_dfs(breakdown_df_shopping_smart, 
                                                                                                                                  shap_df_shopping_smart, 
                                                                                                                                  lime_df_shopping_smart, 
                                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_smart)
print('----------------------------------------------')
print(shap_df_shopping_smart)
print('----------------------------------------------')
print(lime_df_shopping_smart)

In [ ]:
df_final_rf_shopping_smart

### **Results Shopping:**

In [ ]:
df_topfeatures_shopping_comparch = top5_features(df_final_rf_shopping_comparch, 'Shopping - Class: ComputerArchitecture')
df_topfeatures_shopping_emp = top5_features(df_final_rf_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest = top5_features(df_final_rf_shopping_rest, 'Shopping - Class: Restaurant')
df_topfeatures_shopping_shop = top5_features(df_final_rf_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart = top5_features(df_final_rf_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_shopping = pd.concat([df_topfeatures_shopping_comparch, df_topfeatures_shopping_emp, df_topfeatures_shopping_rest, 
                                       df_topfeatures_shopping_shop, df_topfeatures_shopping_smart])
df_topfeatures_shopping

In [ ]:
df_topfeatures_shopping_comparch_rank = top5_rank(df_final_rf_shopping_comparch, 'Shopping - Class: ComputerArchitecture')
df_topfeatures_shopping_emp_rank = top5_rank(df_final_rf_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest_rank = top5_rank(df_final_rf_shopping_rest, 'Shopping  - Class: Restaurant')
df_topfeatures_shopping_shop_rank = top5_rank(df_final_rf_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart_rank = top5_rank(df_final_rf_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_rank_shopping= pd.concat([df_topfeatures_shopping_comparch_rank, df_topfeatures_shopping_emp_rank, 
                                           df_topfeatures_shopping_rest_rank, df_topfeatures_shopping_shop_rank, df_topfeatures_shopping_smart_rank])
df_topfeatures_rank_shopping

In [ ]:
df_topfeatures_shopping_comparch_sign = top5_sign(df_final_rf_shopping_comparch, 'Shopping  - Class: ComputerArchitecture')
df_topfeatures_shopping_emp_sign = top5_sign(df_final_rf_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest_sign = top5_sign(df_final_rf_shopping_rest, 'Shopping  - Class: Restaurant')
df_topfeatures_shopping_shop_sign = top5_sign(df_final_rf_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart_sign = top5_sign(df_final_rf_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_sign_shopping = pd.concat([df_topfeatures_shopping_comparch_sign, df_topfeatures_shopping_emp_sign, 
                                            df_topfeatures_shopping_rest_sign, df_topfeatures_shopping_shop_sign, df_topfeatures_shopping_smart_sign])
df_topfeatures_sign_shopping

In [ ]:
df_topfeatures_shopping_comparch_rank_sign = top5_rank_sign(df_final_rf_shopping_comparch, 'Shopping  - Class: ComputerArchitecture')
df_topfeatures_shopping_emp_rank_sign = top5_rank_sign(df_final_rf_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest_rank_sign = top5_rank_sign(df_final_rf_shopping_rest, 'Shopping  - Class: Restaurant')
df_topfeatures_shopping_shop_rank_sign = top5_rank_sign(df_final_rf_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart_rank_sign = top5_rank_sign(df_final_rf_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_rank_sign_shopping = pd.concat([df_topfeatures_shopping_comparch_rank_sign, df_topfeatures_shopping_emp_rank_sign, 
                                                 df_topfeatures_shopping_rest_rank_sign, df_topfeatures_shopping_shop_rank_sign, 
                                                 df_topfeatures_shopping_smart_rank_sign])
df_topfeatures_rank_sign_shopping

### **SmartHouse:**

In [ ]:
breakdown_df_smarthouse_comparch = rf_exp_results['breakdown_smarthouse'][2].result
shap_df_smarthouse_comparch = rf_exp_results['shap_smarthouse'][2].result
lime_df_smarthouse_comparch = rf_exp_results['lime_smarthouse'][2].result

In [ ]:
breakdown_df_smarthouse_emp = rf_exp_results['breakdown_smarthouse'][4].result
shap_df_smarthouse_emp = rf_exp_results['shap_smarthouse'][4].result
lime_df_smarthouse_emp = rf_exp_results['lime_smarthouse'][4].result

In [ ]:
breakdown_df_smarthouse_rest = rf_exp_results['breakdown_smarthouse'][7].result
shap_df_smarthouse_rest = rf_exp_results['shap_smarthouse'][7].result
lime_df_smarthouse_rest = rf_exp_results['lime_smarthouse'][7].result

In [ ]:
breakdown_df_smarthouse_shop = rf_exp_results['breakdown_smarthouse'][8].result
shap_df_smarthouse_shop = rf_exp_results['shap_smarthouse'][8].result
lime_df_smarthouse_shop = rf_exp_results['lime_smarthouse'][8].result

In [ ]:
breakdown_df_smarthouse_smart = rf_exp_results['breakdown_smarthouse'][9].result
shap_df_smarthouse_smart = rf_exp_results['shap_smarthouse'][9].result
lime_df_smarthouse_smart = rf_exp_results['lime_smarthouse'][9].result

In [ ]:
exp_plots_bd = [rf_exp_results['breakdown_smarthouse'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [rf_exp_results['shap_smarthouse'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  rf_exp_results['lime_smarthouse'][i].plot()

In [ ]:
breakdown_df_smarthouse_comparch, shap_df_smarthouse_comparch, lime_df_smarthouse_comparch, df_final_rf_smarthouse_comparch = preprocess_dfs(breakdown_df_smarthouse_comparch, 
                                                                                                                                      shap_df_smarthouse_comparch, 
                                                                                                                                      lime_df_smarthouse_comparch, 
                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_comparch)
print('----------------------------------------------')
print(shap_df_smarthouse_comparch)
print('----------------------------------------------')
print(lime_df_smarthouse_comparch)

In [ ]:
df_final_rf_smarthouse_comparch

In [ ]:
breakdown_df_smarthouse_emp, shap_df_smarthouse_emp, lime_df_smarthouse_emp, df_final_rf_smarthouse_emp = preprocess_dfs(breakdown_df_smarthouse_emp, 
                                                                                                                  shap_df_smarthouse_emp, 
                                                                                                                  lime_df_smarthouse_emp, 
                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_emp)
print('----------------------------------------------')
print(shap_df_smarthouse_emp)
print('----------------------------------------------')
print(lime_df_smarthouse_emp)

In [ ]:
df_final_rf_smarthouse_emp

In [ ]:
breakdown_df_smarthouse_rest, shap_df_smarthouse_rest, lime_df_smarthouse_rest, df_final_rf_smarthouse_rest = preprocess_dfs(breakdown_df_smarthouse_rest, 
                                                                                                                      shap_df_smarthouse_rest, 
                                                                                                                      lime_df_smarthouse_rest, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_rest)
print('----------------------------------------------')
print(shap_df_smarthouse_rest)
print('----------------------------------------------')
print(lime_df_smarthouse_rest)

In [ ]:
df_final_rf_smarthouse_rest

In [ ]:
breakdown_df_smarthouse_shop, shap_df_smarthouse_shop, lime_df_smarthouse_shop, df_final_rf_smarthouse_shop = preprocess_dfs(breakdown_df_smarthouse_shop, 
                                                                                                                      shap_df_smarthouse_shop, 
                                                                                                                      lime_df_smarthouse_shop, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_shop)
print('----------------------------------------------')
print(shap_df_smarthouse_shop)
print('----------------------------------------------')
print(lime_df_smarthouse_shop)

In [ ]:
df_final_rf_smarthouse_shop

In [ ]:
breakdown_df_smarthouse_smart, shap_df_smarthouse_smart, lime_df_smarthouse_smart, df_final_rf_smarthouse_smart = preprocess_dfs(breakdown_df_smarthouse_smart, 
                                                                                                                          shap_df_smarthouse_smart, 
                                                                                                                          lime_df_smarthouse_smart, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_smart)
print('----------------------------------------------')
print(shap_df_smarthouse_smart)
print('----------------------------------------------')
print(lime_df_smarthouse_smart)

In [ ]:
df_final_rf_smarthouse_smart

### **Results SmartHouse:**

In [ ]:
df_topfeatures_smarthouse_comparch = top5_features(df_final_rf_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp = top5_features(df_final_rf_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest = top5_features(df_final_rf_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop = top5_features(df_final_rf_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart = top5_features(df_final_rf_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch, df_topfeatures_smarthouse_emp, df_topfeatures_smarthouse_rest, 
                                     df_topfeatures_smarthouse_shop, df_topfeatures_smarthouse_smart])
df_topfeatures_smarthouse

In [ ]:
df_topfeatures_smarthouse_comparch_rank = top5_rank(df_final_rf_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp_rank = top5_rank(df_final_rf_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest_rank = top5_rank(df_final_rf_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop_rank = top5_rank(df_final_rf_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart_rank = top5_rank(df_final_rf_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_rank_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch_rank, df_topfeatures_smarthouse_emp_rank, 
                                          df_topfeatures_smarthouse_rest_rank, df_topfeatures_smarthouse_shop_rank, df_topfeatures_smarthouse_smart_rank])
df_topfeatures_rank_smarthouse

In [ ]:
df_topfeatures_smarthouse_comparch_sign = top5_sign(df_final_rf_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp_sign = top5_sign(df_final_rf_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest_sign = top5_sign(df_final_rf_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop_sign = top5_sign(df_final_rf_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart_sign = top5_sign(df_final_rf_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_sign_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch_sign, df_topfeatures_smarthouse_emp_sign, 
                                          df_topfeatures_smarthouse_rest_sign, df_topfeatures_smarthouse_shop_sign, df_topfeatures_smarthouse_smart_sign])
df_topfeatures_sign_smarthouse

In [ ]:
df_topfeatures_smarthouse_comparch_rank_sign = top5_rank_sign(df_final_rf_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp_rank_sign = top5_rank_sign(df_final_rf_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest_rank_sign = top5_rank_sign(df_final_rf_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop_rank_sign = top5_rank_sign(df_final_rf_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart_rank_sign = top5_rank_sign(df_final_rf_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_rank_sign_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch_rank_sign, df_topfeatures_smarthouse_emp_rank_sign, 
                                              df_topfeatures_smarthouse_rest_rank_sign, df_topfeatures_smarthouse_shop_rank_sign, 
                                              df_topfeatures_smarthouse_smart_rank_sign])
df_topfeatures_rank_sign_smarthouse

## KNN:

In [ ]:
exp_list_knn = []

for i in range(len(unique_values)):
    class_value = unique_values[i]
    pf = lambda m, d, i=i: m.predict_proba(d)[:, i]
    e = dx.Explainer(
        model_knn, X_train,
        y_train,
        predict_function=pf,
        label=f'KNNClassifier: class {class_value}',
        verbose=False
    )
    exp_list_knn += [e]

exp_list_knn

In [ ]:
inst_computerarchitecture = X_test.loc[inst_comp_knn]
inst_restaurant = X_test.loc[inst_rest_knn]
inst_shopping = X_test.loc[inst_shop_knn]
inst_employee = X_test.loc[inst_emp_knn]
inst_smarthouse = X_test.loc[inst_smart_knn]

In [ ]:
knn_exp_results = {}

for instance in inst_names:
    breakdown_exp = [exp.predict_parts(eval(f"inst_{instance}"), type='break_down', random_state=42) for exp in exp_list_knn]
    knn_exp_results[f"breakdown_{instance}"] = breakdown_exp

    shap_exp = [exp.predict_parts(eval(f"inst_{instance}"), type='shap', random_state=42) for exp in exp_list_knn]
    knn_exp_results[f"shap_{instance}"] = shap_exp

    lime_exp = [exp.predict_surrogate(eval(f"inst_{instance}"), random_state=42, class_names=[0, unique_values[i]]) for i, exp in enumerate(exp_list_knn)]
    knn_exp_results[f"lime_{instance}"] = lime_exp

### **Computer-architecture:**

In [ ]:
breakdown_df_computerarchitecture_comparch = knn_exp_results['breakdown_computerarchitecture'][2].result
shap_df_computerarchitecture_comparch = knn_exp_results['shap_computerarchitecture'][2].result
lime_df_computerarchitecture_comparch = knn_exp_results['lime_computerarchitecture'][2].result

In [ ]:
breakdown_df_computerarchitecture_emp = knn_exp_results['breakdown_computerarchitecture'][4].result
shap_df_computerarchitecture_emp = knn_exp_results['shap_computerarchitecture'][4].result
lime_df_computerarchitecture_emp = knn_exp_results['lime_computerarchitecture'][4].result

In [ ]:
breakdown_df_computerarchitecture_rest = knn_exp_results['breakdown_computerarchitecture'][7].result
shap_df_computerarchitecture_rest = knn_exp_results['shap_computerarchitecture'][7].result
lime_df_computerarchitecture_rest = knn_exp_results['lime_computerarchitecture'][7].result

In [ ]:
breakdown_df_computerarchitecture_shop = knn_exp_results['breakdown_computerarchitecture'][8].result
shap_df_computerarchitecture_shop = knn_exp_results['shap_computerarchitecture'][8].result
lime_df_computerarchitecture_shop = knn_exp_results['lime_computerarchitecture'][8].result

In [ ]:
breakdown_df_computerarchitecture_smart = knn_exp_results['breakdown_computerarchitecture'][9].result
shap_df_computerarchitecture_smart = knn_exp_results['shap_computerarchitecture'][9].result
lime_df_computerarchitecture_smart = knn_exp_results['lime_computerarchitecture'][9].result

In [ ]:
exp_plots_bd = [knn_exp_results['breakdown_computerarchitecture'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [knn_exp_results['shap_computerarchitecture'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  knn_exp_results['lime_computerarchitecture'][i].plot()

In [ ]:
breakdown_df_computerarchitecture_comparch, shap_df_computerarchitecture_comparch, lime_df_computerarchitecture_comparch, df_final_knn_computerarchitecture_comparch = preprocess_dfs(breakdown_df_computerarchitecture_comparch, 
                                                                                                                                                                                      shap_df_computerarchitecture_comparch, 
                                                                                                                                                                                      lime_df_computerarchitecture_comparch, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_comparch)
print('----------------------------------------------')
print(shap_df_computerarchitecture_comparch)
print('----------------------------------------------')
print(lime_df_computerarchitecture_comparch)

In [ ]:
df_final_knn_computerarchitecture_comparch

In [ ]:
breakdown_df_computerarchitecture_emp, shap_df_computerarchitecture_emp, lime_df_computerarchitecture_emp, df_final_knn_computerarchitecture_emp = preprocess_dfs(breakdown_df_computerarchitecture_emp, 
                                                                                                                                                                                      shap_df_computerarchitecture_emp, 
                                                                                                                                                                                      lime_df_computerarchitecture_emp, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_emp)
print('----------------------------------------------')
print(shap_df_computerarchitecture_emp)
print('----------------------------------------------')
print(lime_df_computerarchitecture_emp)

In [ ]:
df_final_knn_computerarchitecture_emp

In [ ]:
breakdown_df_computerarchitecture_rest, shap_df_computerarchitecture_rest, lime_df_computerarchitecture_rest, df_final_knn_computerarchitecture_rest = preprocess_dfs(breakdown_df_computerarchitecture_rest, 
                                                                                                                                                                                      shap_df_computerarchitecture_rest, 
                                                                                                                                                                                      lime_df_computerarchitecture_rest, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_rest)
print('----------------------------------------------')
print(shap_df_computerarchitecture_rest)
print('----------------------------------------------')
print(lime_df_computerarchitecture_rest)

In [ ]:
df_final_knn_computerarchitecture_rest

In [ ]:
breakdown_df_computerarchitecture_shop, shap_df_computerarchitecture_shop, lime_df_computerarchitecture_shop, df_final_knn_computerarchitecture_shop = preprocess_dfs(breakdown_df_computerarchitecture_shop, 
                                                                                                                                                                                      shap_df_computerarchitecture_shop, 
                                                                                                                                                                                      lime_df_computerarchitecture_shop, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_shop)
print('----------------------------------------------')
print(shap_df_computerarchitecture_shop)
print('----------------------------------------------')
print(lime_df_computerarchitecture_shop)

In [ ]:
df_final_knn_computerarchitecture_shop

In [ ]:
breakdown_df_computerarchitecture_smart, shap_df_computerarchitecture_smart, lime_df_computerarchitecture_smart, df_final_knn_computerarchitecture_smart = preprocess_dfs(breakdown_df_computerarchitecture_smart, 
                                                                                                                                                                                      shap_df_computerarchitecture_smart, 
                                                                                                                                                                                      lime_df_computerarchitecture_smart, 
                                                                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_computerarchitecture_smart)
print('----------------------------------------------')
print(shap_df_computerarchitecture_smart)
print('----------------------------------------------')
print(lime_df_computerarchitecture_smart)

In [ ]:
df_final_knn_computerarchitecture_smart

### **Results Computer-architecture:**

In [ ]:
df_topfeatures_comparch_comparch = top5_features(df_final_knn_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp = top5_features(df_final_knn_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest = top5_features(df_final_knn_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop = top5_features(df_final_knn_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart = top5_features(df_final_knn_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch, df_topfeatures_comparch_emp, df_topfeatures_comparch_rest, 
                                                 df_topfeatures_comparch_shop, df_topfeatures_comparch_smart])
df_topfeatures_computerarchitecture

In [ ]:
df_topfeatures_comparch_comparch_rank = top5_rank(df_final_knn_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp_rank = top5_rank(df_final_knn_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest_rank = top5_rank(df_final_knn_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop_rank = top5_rank(df_final_knn_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart_rank = top5_rank(df_final_knn_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_rank_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch_rank, df_topfeatures_comparch_emp_rank, 
                                                      df_topfeatures_comparch_rest_rank, df_topfeatures_comparch_shop_rank, df_topfeatures_comparch_smart_rank])
df_topfeatures_rank_computerarchitecture

In [ ]:
df_topfeatures_comparch_comparch_sign = top5_sign(df_final_knn_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp_sign = top5_sign(df_final_knn_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest_sign = top5_sign(df_final_knn_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop_sign = top5_sign(df_final_knn_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart_sign = top5_sign(df_final_knn_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_sign_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch_sign, df_topfeatures_comparch_emp_sign, 
                                                      df_topfeatures_comparch_rest_sign, df_topfeatures_comparch_shop_sign, df_topfeatures_comparch_smart_sign])
df_topfeatures_sign_computerarchitecture

In [ ]:
df_topfeatures_comparch_comparch_rank_sign = top5_rank_sign(df_final_knn_computerarchitecture_comparch, 'ComputerArchitecture - Class: ComputerArchitecture')
df_topfeatures_comparch_emp_rank_sign = top5_rank_sign(df_final_knn_computerarchitecture_emp, 'ComputerArchitecture - Class: Employee')
df_topfeatures_comparch_rest_rank_sign = top5_rank_sign(df_final_knn_computerarchitecture_rest, 'ComputerArchitecture - Class: Restaurant')
df_topfeatures_comparch_shop_rank_sign = top5_rank_sign(df_final_knn_computerarchitecture_shop, 'ComputerArchitecture - Class: Shopping')
df_topfeatures_comparch_smart_rank_sign = top5_rank_sign(df_final_knn_computerarchitecture_smart, 'ComputerArchitecture - Class: Smarthouse')

df_topfeatures_rank_sign_computerarchitecture = pd.concat([df_topfeatures_comparch_comparch_rank_sign, df_topfeatures_comparch_emp_rank_sign, 
                                                           df_topfeatures_comparch_rest_rank_sign, df_topfeatures_comparch_shop_rank_sign, 
                                                           df_topfeatures_comparch_smart_rank_sign])
df_topfeatures_rank_sign_computerarchitecture

### **Employee:**

In [ ]:
breakdown_df_employee_comparch = knn_exp_results['breakdown_employee'][2].result
shap_df_employee_comparch = knn_exp_results['shap_employee'][2].result
lime_df_employee_comparch = knn_exp_results['lime_employee'][2].result

In [ ]:
breakdown_df_employee_emp = knn_exp_results['breakdown_employee'][4].result
shap_df_employee_emp = knn_exp_results['shap_employee'][4].result
lime_df_employee_emp = knn_exp_results['lime_employee'][4].result

In [ ]:
breakdown_df_employee_rest = knn_exp_results['breakdown_employee'][7].result
shap_df_employee_rest = knn_exp_results['shap_employee'][7].result
lime_df_employee_rest = knn_exp_results['lime_employee'][7].result

In [ ]:
breakdown_df_employee_shop = knn_exp_results['breakdown_employee'][8].result
shap_df_employee_shop = knn_exp_results['shap_employee'][8].result
lime_df_employee_shop = knn_exp_results['lime_employee'][8].result

In [ ]:
breakdown_df_employee_smart = knn_exp_results['breakdown_employee'][9].result
shap_df_employee_smart = knn_exp_results['shap_employee'][9].result
lime_df_employee_smart = knn_exp_results['lime_employee'][9].result

In [ ]:
exp_plots_bd = [knn_exp_results['breakdown_employee'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [knn_exp_results['shap_employee'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  knn_exp_results['lime_employee'][i].plot()

In [ ]:
breakdown_df_employee_comparch, shap_df_employee_comparch, lime_df_employee_comparch, df_final_knn_employee_comparch = preprocess_dfs(breakdown_df_employee_comparch, 
                                                                                                                                      shap_df_employee_comparch, 
                                                                                                                                      lime_df_employee_comparch, 
                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_comparch)
print('----------------------------------------------')
print(shap_df_employee_comparch)
print('----------------------------------------------')
print(lime_df_employee_comparch)

In [ ]:
df_final_knn_employee_comparch

In [ ]:
breakdown_df_employee_emp, shap_df_employee_emp, lime_df_employee_emp, df_final_knn_employee_emp = preprocess_dfs(breakdown_df_employee_emp, 
                                                                                                                  shap_df_employee_emp, 
                                                                                                                  lime_df_employee_emp, 
                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_emp)
print('----------------------------------------------')
print(shap_df_employee_emp)
print('----------------------------------------------')
print(lime_df_employee_emp)

In [ ]:
df_final_knn_employee_emp

In [ ]:
breakdown_df_employee_rest, shap_df_employee_rest, lime_df_employee_rest, df_final_knn_employee_rest = preprocess_dfs(breakdown_df_employee_rest, 
                                                                                                                      shap_df_employee_rest, 
                                                                                                                      lime_df_employee_rest, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_rest)
print('----------------------------------------------')
print(shap_df_employee_rest)
print('----------------------------------------------')
print(lime_df_employee_rest)

In [ ]:
df_final_knn_employee_rest

In [ ]:
breakdown_df_employee_shop, shap_df_employee_shop, lime_df_employee_shop, df_final_knn_employee_shop = preprocess_dfs(breakdown_df_employee_shop, 
                                                                                                                      shap_df_employee_shop, 
                                                                                                                      lime_df_employee_shop, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_shop)
print('----------------------------------------------')
print(shap_df_employee_shop)
print('----------------------------------------------')
print(lime_df_employee_shop)

In [ ]:
df_final_knn_employee_shop

In [ ]:
breakdown_df_employee_smart, shap_df_employee_smart, lime_df_employee_smart, df_final_knn_employee_smart = preprocess_dfs(breakdown_df_employee_smart, 
                                                                                                                          shap_df_employee_smart, 
                                                                                                                          lime_df_employee_smart, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_employee_smart)
print('----------------------------------------------')
print(shap_df_employee_smart)
print('----------------------------------------------')
print(lime_df_employee_smart)

In [ ]:
df_final_knn_employee_smart

### **Results Employee:**

In [ ]:
df_topfeatures_employee_comparch = top5_features(df_final_knn_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp = top5_features(df_final_knn_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest = top5_features(df_final_knn_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop = top5_features(df_final_knn_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart = top5_features(df_final_knn_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_employee = pd.concat([df_topfeatures_employee_comparch, df_topfeatures_employee_emp, df_topfeatures_employee_rest, 
                                     df_topfeatures_employee_shop, df_topfeatures_employee_smart])
df_topfeatures_employee

In [ ]:
df_topfeatures_employee_comparch_rank = top5_rank(df_final_knn_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp_rank = top5_rank(df_final_knn_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest_rank = top5_rank(df_final_knn_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop_rank = top5_rank(df_final_knn_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart_rank = top5_rank(df_final_knn_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_rank_employee = pd.concat([df_topfeatures_employee_comparch_rank, df_topfeatures_employee_emp_rank, 
                                          df_topfeatures_employee_rest_rank, df_topfeatures_employee_shop_rank, df_topfeatures_employee_smart_rank])
df_topfeatures_rank_employee

In [ ]:
df_topfeatures_employee_comparch_sign = top5_sign(df_final_knn_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp_sign = top5_sign(df_final_knn_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest_sign = top5_sign(df_final_knn_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop_sign = top5_sign(df_final_knn_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart_sign = top5_sign(df_final_knn_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_sign_employee = pd.concat([df_topfeatures_employee_comparch_sign, df_topfeatures_employee_emp_sign, 
                                          df_topfeatures_employee_rest_sign, df_topfeatures_employee_shop_sign, df_topfeatures_employee_smart_sign])
df_topfeatures_sign_employee

In [ ]:
df_topfeatures_employee_comparch_rank_sign = top5_rank_sign(df_final_knn_employee_comparch, 'Employee - Class: ComputerArchitecture')
df_topfeatures_employee_emp_rank_sign = top5_rank_sign(df_final_knn_employee_emp, 'Employee - Class: Employee')
df_topfeatures_employee_rest_rank_sign = top5_rank_sign(df_final_knn_employee_rest, 'Employee - Class: Restaurant')
df_topfeatures_employee_shop_rank_sign = top5_rank_sign(df_final_knn_employee_shop, 'Employee - Class: Shopping')
df_topfeatures_employee_smart_rank_sign = top5_rank_sign(df_final_knn_employee_smart, 'Employee - Class: Smarthouse')

df_topfeatures_rank_sign_employee = pd.concat([df_topfeatures_employee_comparch_rank_sign, df_topfeatures_employee_emp_rank_sign, 
                                              df_topfeatures_employee_rest_rank_sign, df_topfeatures_employee_shop_rank_sign, 
                                              df_topfeatures_employee_smart_rank_sign])
df_topfeatures_rank_sign_employee

### **Restaurant:**

In [ ]:
breakdown_df_restaurant_comparch = knn_exp_results['breakdown_restaurant'][2].result
shap_df_restaurant_comparch = knn_exp_results['shap_restaurant'][2].result
lime_df_restaurant_comparch = knn_exp_results['lime_restaurant'][2].result

In [ ]:
breakdown_df_restaurant_emp = knn_exp_results['breakdown_restaurant'][4].result
shap_df_restaurant_emp = knn_exp_results['shap_restaurant'][4].result
lime_df_restaurant_emp = knn_exp_results['lime_restaurant'][4].result

In [ ]:
breakdown_df_restaurant_rest = knn_exp_results['breakdown_restaurant'][7].result
shap_df_restaurant_rest = knn_exp_results['shap_restaurant'][7].result
lime_df_restaurant_rest = knn_exp_results['lime_restaurant'][7].result

In [ ]:
breakdown_df_restaurant_shop = knn_exp_results['breakdown_restaurant'][8].result
shap_df_restaurant_shop = knn_exp_results['shap_restaurant'][8].result
lime_df_restaurant_shop = knn_exp_results['lime_restaurant'][8].result

In [ ]:
breakdown_df_restaurant_smart = knn_exp_results['breakdown_restaurant'][9].result
shap_df_restaurant_smart = knn_exp_results['shap_restaurant'][9].result
lime_df_restaurant_smart = knn_exp_results['lime_restaurant'][9].result

In [ ]:
exp_plots_bd = [knn_exp_results['breakdown_restaurant'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [knn_exp_results['shap_restaurant'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  knn_exp_results['lime_restaurant'][i].plot()

In [ ]:
breakdown_df_restaurant_comparch, shap_df_restaurant_comparch, lime_df_restaurant_comparch, df_final_knn_restaurant_comparch = preprocess_dfs(breakdown_df_restaurant_comparch, 
                                                                                                                                              shap_df_restaurant_comparch, 
                                                                                                                                              lime_df_restaurant_comparch, 
                                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_comparch)
print('----------------------------------------------')
print(shap_df_restaurant_comparch)
print('----------------------------------------------')
print(lime_df_restaurant_comparch)

In [ ]:
df_final_knn_restaurant_comparch

In [ ]:
breakdown_df_restaurant_emp, shap_df_restaurant_emp, lime_df_restaurant_emp, df_final_knn_restaurant_emp = preprocess_dfs(breakdown_df_restaurant_emp, 
                                                                                                                          shap_df_restaurant_emp, 
                                                                                                                          lime_df_restaurant_emp, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_emp)
print('----------------------------------------------')
print(shap_df_restaurant_emp)
print('----------------------------------------------')
print(lime_df_restaurant_emp)

In [ ]:
df_final_knn_restaurant_emp

In [ ]:
breakdown_df_restaurant_rest, shap_df_restaurant_rest, lime_df_restaurant_rest, df_final_knn_restaurant_rest = preprocess_dfs(breakdown_df_restaurant_rest, 
                                                                                                                              shap_df_restaurant_rest, 
                                                                                                                              lime_df_restaurant_rest, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_rest)
print('----------------------------------------------')
print(shap_df_restaurant_rest)
print('----------------------------------------------')
print(lime_df_restaurant_rest)

In [ ]:
df_final_knn_restaurant_rest

In [ ]:
breakdown_df_restaurant_shop, shap_df_restaurant_shop, lime_df_restaurant_shop, df_final_knn_restaurant_shop = preprocess_dfs(breakdown_df_restaurant_shop, 
                                                                                                                              shap_df_restaurant_shop, 
                                                                                                                              lime_df_restaurant_shop, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_shop)
print('----------------------------------------------')
print(shap_df_restaurant_shop)
print('----------------------------------------------')
print(lime_df_restaurant_shop)

In [ ]:
df_final_knn_restaurant_shop

In [ ]:
breakdown_df_restaurant_smart, shap_df_restaurant_smart, lime_df_restaurant_smart, df_final_knn_restaurant_smart = preprocess_dfs(breakdown_df_restaurant_smart, 
                                                                                                                                  shap_df_restaurant_smart, 
                                                                                                                                  lime_df_restaurant_smart, 
                                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_restaurant_smart)
print('----------------------------------------------')
print(shap_df_restaurant_smart)
print('----------------------------------------------')
print(lime_df_restaurant_smart)

In [ ]:
df_final_knn_restaurant_smart

### **Results Restaurant:**

In [ ]:
df_topfeatures_restaurant_comparch = top5_features(df_final_knn_restaurant_comparch, 'Restaurant - Class: ComputerArchitecture')
df_topfeatures_restaurant_emp = top5_features(df_final_knn_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest = top5_features(df_final_knn_restaurant_rest, 'Restaurant- Class: Restaurant')
df_topfeatures_restaurant_shop = top5_features(df_final_knn_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart = top5_features(df_final_knn_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_restaurant = pd.concat([df_topfeatures_restaurant_comparch, df_topfeatures_restaurant_emp, df_topfeatures_restaurant_rest, 
                                       df_topfeatures_restaurant_shop, df_topfeatures_restaurant_smart])
df_topfeatures_restaurant

In [ ]:
df_topfeatures_restaurant_comparch_rank = top5_rank(df_final_knn_restaurant_comparch, 'Restaurant- Class: ComputerArchitecture')
df_topfeatures_restaurant_emp_rank = top5_rank(df_final_knn_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest_rank = top5_rank(df_final_knn_restaurant_rest, 'Restaurant - Class: Restaurant')
df_topfeatures_restaurant_shop_rank = top5_rank(df_final_knn_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart_rank = top5_rank(df_final_knn_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_rank_restaurant= pd.concat([df_topfeatures_restaurant_comparch_rank, df_topfeatures_restaurant_emp_rank, 
                                           df_topfeatures_restaurant_rest_rank, df_topfeatures_restaurant_shop_rank, df_topfeatures_restaurant_smart_rank])
df_topfeatures_rank_restaurant

In [ ]:
df_topfeatures_restaurant_comparch_sign = top5_sign(df_final_knn_restaurant_comparch, 'Restaurant - Class: ComputerArchitecture')
df_topfeatures_restaurant_emp_sign = top5_sign(df_final_knn_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest_sign = top5_sign(df_final_knn_restaurant_rest, 'Restaurant - Class: Restaurant')
df_topfeatures_restaurant_shop_sign = top5_sign(df_final_knn_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart_sign = top5_sign(df_final_knn_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_sign_restaurant = pd.concat([df_topfeatures_restaurant_comparch_sign, df_topfeatures_restaurant_emp_sign, 
                                            df_topfeatures_restaurant_rest_sign, df_topfeatures_restaurant_shop_sign, df_topfeatures_restaurant_smart_sign])
df_topfeatures_sign_restaurant

In [ ]:
df_topfeatures_restaurant_comparch_rank_sign = top5_rank_sign(df_final_knn_restaurant_comparch, 'Restaurant - Class: ComputerArchitecture')
df_topfeatures_restaurant_emp_rank_sign = top5_rank_sign(df_final_knn_restaurant_emp, 'Restaurant - Class: Employee')
df_topfeatures_restaurant_rest_rank_sign = top5_rank_sign(df_final_knn_restaurant_rest, 'Restaurant - Class: Restaurant')
df_topfeatures_restaurant_shop_rank_sign = top5_rank_sign(df_final_knn_restaurant_shop, 'Restaurant - Class: Shopping')
df_topfeatures_restaurant_smart_rank_sign = top5_rank_sign(df_final_knn_restaurant_smart, 'Restaurant - Class: Smarthouse')

df_topfeatures_rank_sign_restaurant = pd.concat([df_topfeatures_restaurant_comparch_rank_sign, df_topfeatures_restaurant_emp_rank_sign, 
                                                 df_topfeatures_restaurant_rest_rank_sign, df_topfeatures_restaurant_shop_rank_sign, 
                                                 df_topfeatures_restaurant_smart_rank_sign])
df_topfeatures_rank_sign_restaurant

### **Shopping:**

In [ ]:
breakdown_df_shopping_comparch = knn_exp_results['breakdown_shopping'][2].result
shap_df_shopping_comparch = knn_exp_results['shap_shopping'][2].result
lime_df_shopping_comparch = knn_exp_results['lime_shopping'][2].result

In [ ]:
breakdown_df_shopping_emp = knn_exp_results['breakdown_shopping'][4].result
shap_df_shopping_emp = knn_exp_results['shap_shopping'][4].result
lime_df_shopping_emp = knn_exp_results['lime_shopping'][4].result

In [ ]:
breakdown_df_shopping_rest = knn_exp_results['breakdown_shopping'][7].result
shap_df_shopping_rest = knn_exp_results['shap_shopping'][7].result
lime_df_shopping_rest = knn_exp_results['lime_shopping'][7].result

In [ ]:
breakdown_df_shopping_shop = knn_exp_results['breakdown_shopping'][8].result
shap_df_shopping_shop = knn_exp_results['shap_shopping'][8].result
lime_df_shopping_shop = knn_exp_results['lime_shopping'][8].result

In [ ]:
breakdown_df_shopping_smart = knn_exp_results['breakdown_shopping'][9].result
shap_df_shopping_smart = knn_exp_results['shap_shopping'][9].result
lime_df_shopping_smart = knn_exp_results['lime_shopping'][9].result

In [ ]:
exp_plots_bd = [knn_exp_results['breakdown_shopping'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [knn_exp_results['shap_shopping'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  knn_exp_results['lime_shopping'][i].plot()

In [ ]:
breakdown_df_shopping_comparch, shap_df_shopping_comparch, lime_df_shopping_comparch, df_final_knn_shopping_comparch = preprocess_dfs(breakdown_df_shopping_comparch, 
                                                                                                                                              shap_df_shopping_comparch, 
                                                                                                                                              lime_df_shopping_comparch, 
                                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_comparch)
print('----------------------------------------------')
print(shap_df_shopping_comparch)
print('----------------------------------------------')
print(lime_df_shopping_comparch)

In [ ]:
df_final_knn_shopping_comparch

In [ ]:
breakdown_df_shopping_emp, shap_df_shopping_emp, lime_df_shopping_emp, df_final_knn_shopping_emp = preprocess_dfs(breakdown_df_shopping_emp, 
                                                                                                                          shap_df_shopping_emp, 
                                                                                                                          lime_df_shopping_emp, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_emp)
print('----------------------------------------------')
print(shap_df_shopping_emp)
print('----------------------------------------------')
print(lime_df_shopping_emp)

In [ ]:
df_final_knn_shopping_emp

In [ ]:
breakdown_df_shopping_rest, shap_df_shopping_rest, lime_df_shopping_rest, df_final_knn_shopping_rest = preprocess_dfs(breakdown_df_shopping_rest, 
                                                                                                                              shap_df_shopping_rest, 
                                                                                                                              lime_df_shopping_rest, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_rest)
print('----------------------------------------------')
print(shap_df_shopping_rest)
print('----------------------------------------------')
print(lime_df_shopping_rest)

In [ ]:
df_final_knn_shopping_rest

In [ ]:
breakdown_df_shopping_shop, shap_df_shopping_shop, lime_df_shopping_shop, df_final_knn_shopping_shop = preprocess_dfs(breakdown_df_shopping_shop, 
                                                                                                                              shap_df_shopping_shop, 
                                                                                                                              lime_df_shopping_shop, 
                                                                                                                              num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_shop)
print('----------------------------------------------')
print(shap_df_shopping_shop)
print('----------------------------------------------')
print(lime_df_shopping_shop)

In [ ]:
df_final_knn_shopping_shop

In [ ]:
breakdown_df_shopping_smart, shap_df_shopping_smart, lime_df_shopping_smart, df_final_knn_shopping_smart = preprocess_dfs(breakdown_df_shopping_smart, 
                                                                                                                                  shap_df_shopping_smart, 
                                                                                                                                  lime_df_shopping_smart, 
                                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_shopping_smart)
print('----------------------------------------------')
print(shap_df_shopping_smart)
print('----------------------------------------------')
print(lime_df_shopping_smart)

In [ ]:
df_final_knn_shopping_smart

### **Results Shopping:**

In [ ]:
df_topfeatures_shopping_comparch = top5_features(df_final_knn_shopping_comparch, 'Shopping - Class: ComputerArchitecture')
df_topfeatures_shopping_emp = top5_features(df_final_knn_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest = top5_features(df_final_knn_shopping_rest, 'Shopping - Class: Restaurant')
df_topfeatures_shopping_shop = top5_features(df_final_knn_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart = top5_features(df_final_knn_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_shopping = pd.concat([df_topfeatures_shopping_comparch, df_topfeatures_shopping_emp, df_topfeatures_shopping_rest, 
                                       df_topfeatures_shopping_shop, df_topfeatures_shopping_smart])
df_topfeatures_shopping

In [ ]:
df_topfeatures_shopping_comparch_rank = top5_rank(df_final_knn_shopping_comparch, 'Shopping - Class: ComputerArchitecture')
df_topfeatures_shopping_emp_rank = top5_rank(df_final_knn_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest_rank = top5_rank(df_final_knn_shopping_rest, 'Shopping  - Class: Restaurant')
df_topfeatures_shopping_shop_rank = top5_rank(df_final_knn_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart_rank = top5_rank(df_final_knn_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_rank_shopping= pd.concat([df_topfeatures_shopping_comparch_rank, df_topfeatures_shopping_emp_rank, 
                                           df_topfeatures_shopping_rest_rank, df_topfeatures_shopping_shop_rank, df_topfeatures_shopping_smart_rank])
df_topfeatures_rank_shopping

In [ ]:
df_topfeatures_shopping_comparch_sign = top5_sign(df_final_knn_shopping_comparch, 'Shopping  - Class: ComputerArchitecture')
df_topfeatures_shopping_emp_sign = top5_sign(df_final_knn_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest_sign = top5_sign(df_final_knn_shopping_rest, 'Shopping  - Class: Restaurant')
df_topfeatures_shopping_shop_sign = top5_sign(df_final_knn_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart_sign = top5_sign(df_final_knn_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_sign_shopping = pd.concat([df_topfeatures_shopping_comparch_sign, df_topfeatures_shopping_emp_sign, 
                                            df_topfeatures_shopping_rest_sign, df_topfeatures_shopping_shop_sign, df_topfeatures_shopping_smart_sign])
df_topfeatures_sign_shopping

In [ ]:
df_topfeatures_shopping_comparch_rank_sign = top5_rank_sign(df_final_knn_shopping_comparch, 'Shopping  - Class: ComputerArchitecture')
df_topfeatures_shopping_emp_rank_sign = top5_rank_sign(df_final_knn_shopping_emp, 'Shopping  - Class: Employee')
df_topfeatures_shopping_rest_rank_sign = top5_rank_sign(df_final_knn_shopping_rest, 'Shopping  - Class: Restaurant')
df_topfeatures_shopping_shop_rank_sign = top5_rank_sign(df_final_knn_shopping_shop, 'Shopping  - Class: Shopping')
df_topfeatures_shopping_smart_rank_sign = top5_rank_sign(df_final_knn_shopping_smart, 'Shopping  - Class: Smarthouse')

df_topfeatures_rank_sign_shopping = pd.concat([df_topfeatures_shopping_comparch_rank_sign, df_topfeatures_shopping_emp_rank_sign, 
                                                 df_topfeatures_shopping_rest_rank_sign, df_topfeatures_shopping_shop_rank_sign, 
                                                 df_topfeatures_shopping_smart_rank_sign])
df_topfeatures_rank_sign_shopping

### **SmartHouse:**

In [ ]:
breakdown_df_smarthouse_comparch = knn_exp_results['breakdown_smarthouse'][2].result
shap_df_smarthouse_comparch = knn_exp_results['shap_smarthouse'][2].result
lime_df_smarthouse_comparch = knn_exp_results['lime_smarthouse'][2].result

In [ ]:
breakdown_df_smarthouse_emp = knn_exp_results['breakdown_smarthouse'][4].result
shap_df_smarthouse_emp = knn_exp_results['shap_smarthouse'][4].result
lime_df_smarthouse_emp = knn_exp_results['lime_smarthouse'][4].result

In [ ]:
breakdown_df_smarthouse_rest = knn_exp_results['breakdown_smarthouse'][7].result
shap_df_smarthouse_rest = knn_exp_results['shap_smarthouse'][7].result
lime_df_smarthouse_rest = knn_exp_results['lime_smarthouse'][7].result

In [ ]:
breakdown_df_smarthouse_shop = knn_exp_results['breakdown_smarthouse'][8].result
shap_df_smarthouse_shop = knn_exp_results['shap_smarthouse'][8].result
lime_df_smarthouse_shop = knn_exp_results['lime_smarthouse'][8].result

In [ ]:
breakdown_df_smarthouse_smart = knn_exp_results['breakdown_smarthouse'][9].result
shap_df_smarthouse_smart = knn_exp_results['shap_smarthouse'][9].result
lime_df_smarthouse_smart = knn_exp_results['lime_smarthouse'][9].result

In [ ]:
exp_plots_bd = [knn_exp_results['breakdown_smarthouse'][i] for i in class_index]
exp_plots_bd[0].plot(exp_plots_bd[1:])

In [ ]:
exp_plots_shap = [knn_exp_results['shap_smarthouse'][i] for i in class_index]
exp_plots_shap[0].plot(exp_plots_shap[1:])

In [ ]:
for i in class_index:
  knn_exp_results['lime_smarthouse'][i].plot()

In [ ]:
breakdown_df_smarthouse_comparch, shap_df_smarthouse_comparch, lime_df_smarthouse_comparch, df_final_knn_smarthouse_comparch = preprocess_dfs(breakdown_df_smarthouse_comparch, 
                                                                                                                                      shap_df_smarthouse_comparch, 
                                                                                                                                      lime_df_smarthouse_comparch, 
                                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_comparch)
print('----------------------------------------------')
print(shap_df_smarthouse_comparch)
print('----------------------------------------------')
print(lime_df_smarthouse_comparch)

In [ ]:
df_final_knn_smarthouse_comparch

In [ ]:
breakdown_df_smarthouse_emp, shap_df_smarthouse_emp, lime_df_smarthouse_emp, df_final_knn_smarthouse_emp = preprocess_dfs(breakdown_df_smarthouse_emp, 
                                                                                                                  shap_df_smarthouse_emp, 
                                                                                                                  lime_df_smarthouse_emp, 
                                                                                                                  num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_emp)
print('----------------------------------------------')
print(shap_df_smarthouse_emp)
print('----------------------------------------------')
print(lime_df_smarthouse_emp)

In [ ]:
df_final_knn_smarthouse_emp

In [ ]:
breakdown_df_smarthouse_rest, shap_df_smarthouse_rest, lime_df_smarthouse_rest, df_final_knn_smarthouse_rest = preprocess_dfs(breakdown_df_smarthouse_rest, 
                                                                                                                      shap_df_smarthouse_rest, 
                                                                                                                      lime_df_smarthouse_rest, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_rest)
print('----------------------------------------------')
print(shap_df_smarthouse_rest)
print('----------------------------------------------')
print(lime_df_smarthouse_rest)

In [ ]:
df_final_knn_smarthouse_rest

In [ ]:
breakdown_df_smarthouse_shop, shap_df_smarthouse_shop, lime_df_smarthouse_shop, df_final_knn_smarthouse_shop = preprocess_dfs(breakdown_df_smarthouse_shop, 
                                                                                                                      shap_df_smarthouse_shop, 
                                                                                                                      lime_df_smarthouse_shop, 
                                                                                                                      num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_shop)
print('----------------------------------------------')
print(shap_df_smarthouse_shop)
print('----------------------------------------------')
print(lime_df_smarthouse_shop)

In [ ]:
df_final_knn_smarthouse_shop

In [ ]:
breakdown_df_smarthouse_smart, shap_df_smarthouse_smart, lime_df_smarthouse_smart, df_final_knn_smarthouse_smart = preprocess_dfs(breakdown_df_smarthouse_smart, 
                                                                                                                          shap_df_smarthouse_smart, 
                                                                                                                          lime_df_smarthouse_smart, 
                                                                                                                          num_features, top_num_features)
print('----------------------------------------------')
print(breakdown_df_smarthouse_smart)
print('----------------------------------------------')
print(shap_df_smarthouse_smart)
print('----------------------------------------------')
print(lime_df_smarthouse_smart)

In [ ]:
df_final_knn_smarthouse_smart

### **Results SmartHouse:**

In [ ]:
df_topfeatures_smarthouse_comparch = top5_features(df_final_knn_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp = top5_features(df_final_knn_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest = top5_features(df_final_knn_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop = top5_features(df_final_knn_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart = top5_features(df_final_knn_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch, df_topfeatures_smarthouse_emp, df_topfeatures_smarthouse_rest, 
                                     df_topfeatures_smarthouse_shop, df_topfeatures_smarthouse_smart])
df_topfeatures_smarthouse

In [ ]:
df_topfeatures_smarthouse_comparch_rank = top5_rank(df_final_knn_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp_rank = top5_rank(df_final_knn_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest_rank = top5_rank(df_final_knn_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop_rank = top5_rank(df_final_knn_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart_rank = top5_rank(df_final_knn_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_rank_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch_rank, df_topfeatures_smarthouse_emp_rank, 
                                          df_topfeatures_smarthouse_rest_rank, df_topfeatures_smarthouse_shop_rank, df_topfeatures_smarthouse_smart_rank])
df_topfeatures_rank_smarthouse

In [ ]:
df_topfeatures_smarthouse_comparch_sign = top5_sign(df_final_knn_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp_sign = top5_sign(df_final_knn_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest_sign = top5_sign(df_final_knn_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop_sign = top5_sign(df_final_knn_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart_sign = top5_sign(df_final_knn_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_sign_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch_sign, df_topfeatures_smarthouse_emp_sign, 
                                          df_topfeatures_smarthouse_rest_sign, df_topfeatures_smarthouse_shop_sign, df_topfeatures_smarthouse_smart_sign])
df_topfeatures_sign_smarthouse

In [ ]:
df_topfeatures_smarthouse_comparch_rank_sign = top5_rank_sign(df_final_knn_smarthouse_comparch, 'SmartHouse - Class: ComputerArchitecture')
df_topfeatures_smarthouse_emp_rank_sign = top5_rank_sign(df_final_knn_smarthouse_emp, 'SmartHouse - Class: Employee')
df_topfeatures_smarthouse_rest_rank_sign = top5_rank_sign(df_final_knn_smarthouse_rest, 'SmartHouse - Class: Restaurant')
df_topfeatures_smarthouse_shop_rank_sign = top5_rank_sign(df_final_knn_smarthouse_shop, 'SmartHouse - Class: Shopping')
df_topfeatures_smarthouse_smart_rank_sign = top5_rank_sign(df_final_knn_smarthouse_smart, 'SmartHouse - Class: Smarthouse')

df_topfeatures_rank_sign_smarthouse = pd.concat([df_topfeatures_smarthouse_comparch_rank_sign, df_topfeatures_smarthouse_emp_rank_sign, 
                                              df_topfeatures_smarthouse_rest_rank_sign, df_topfeatures_smarthouse_shop_rank_sign, 
                                              df_topfeatures_smarthouse_smart_rank_sign])
df_topfeatures_rank_sign_smarthouse